In [1]:
# %% [markdown]
# ## 1. 导入库和设置参数
# %%
import backtrader as bt
import pandas as pd
import numpy as np
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置回测参数
start_date = datetime(2021, 7, 1)
end_date = datetime(2025, 7, 1)
initial_cash = 100000

In [2]:
# %% [markdown]
# ## 2. 定义ETF列表
# %%
intl_etfs = {
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'
}

china_etfs = {
    '159980': '有色金属ETF',
    '513010': '恒生科技',
    '159892': '恒生医药',
    '159934': '黄金ETF',
    '159985': '豆粕ETF',
    '510880': '红利ETF',
    '516780': '稀土ETF'
}

# 合并所有ETF代码
all_tickers = list(intl_etfs.keys()) + list(china_etfs.keys())

In [3]:
# %% [markdown]
# ## 3. 数据准备函数
# %%
def prepare_data(tickers, start_date, end_date):
    """准备Backtrader兼容的数据"""
    datafeeds = {}
    
    for ticker in tickers:
        print(f"正在获取 {ticker} 数据...")
        try:
            # 判断是国际ETF还是中国ETF
            if ticker in intl_etfs:
                # 使用yfinance获取数据
                df = yf.download(ticker, start=start_date, end=end_date)
                if df.empty:
                    print(f"警告: {ticker} 无有效数据")
                    continue
                df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
                df.columns = ['open', 'high', 'low', 'close', 'volume']
            else:
                # 使用akshare获取中国ETF数据
                df = ak.fund_etf_hist_em(
                    symbol=ticker, 
                    period="daily", 
                    start_date=start_date.strftime("%Y%m%d"),
                    end_date=end_date.strftime("%Y%m%d"), 
                    adjust="hfq"
                )
                if df.empty:
                    print(f"警告: {ticker} 无有效数据")
                    continue
                df = df.rename(columns={
                    '日期':'date', 
                    '开盘':'open', 
                    '最高':'high',
                    '最低':'low', 
                    '收盘':'close', 
                    '成交量':'volume'
                })
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
            
            # 确保数据完整性
            df = df.dropna()
            if len(df) < 20:  # 至少20个交易日数据
                print(f"警告: {ticker} 数据量不足")
                continue
                
            # 转换为Backtrader数据格式
            data = bt.feeds.PandasData(dataname=df)
            datafeeds[ticker] = data
            print(f"{ticker} 数据准备完成，共{len(df)}条数据")
        except Exception as e:
            print(f"获取 {ticker} 数据时出错: {str(e)}")
            continue
    
    return datafeeds

In [4]:
# %% [markdown]
# ## 4. 准备数据
# %%
print("开始准备数据...")
datafeeds = prepare_data(all_tickers, start_date, end_date)
print(f"\n成功获取 {len(datafeeds)} 只ETF数据")


开始准备数据...
正在获取 QQQ 数据...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


QQQ 数据准备完成，共1003条数据
正在获取 SPY 数据...


[*********************100%***********************]  1 of 1 completed


SPY 数据准备完成，共1003条数据
正在获取 159980 数据...


  0%|          | 0/11 [00:00<?, ?it/s]

159980 数据准备完成，共969条数据
正在获取 513010 数据...
513010 数据准备完成，共969条数据
正在获取 159892 数据...
159892 数据准备完成，共898条数据
正在获取 159934 数据...
159934 数据准备完成，共969条数据
正在获取 159985 数据...
159985 数据准备完成，共969条数据
正在获取 510880 数据...
510880 数据准备完成，共969条数据
正在获取 516780 数据...
516780 数据准备完成，共969条数据

成功获取 9 只ETF数据


In [5]:
# %% [markdown]
# ## 5. 优化后的策略类
# %%
class OptimizedTop4Strategy(bt.Strategy):
    params = (
        ('top_n', 4),          # 选择前4名
        ('weight', 0.25),      # 每只ETF权重25%
        ('rebalance_months', 3), # 每3个月调仓
    )

    def __init__(self):
        self.last_rebalance = None
        self.performance = []
        
        # 记录每个数据的开始日期
        self.data_start_dates = {data._name: data.datetime.date(0) for data in self.datas}
        print("各ETF数据起始日期:")
        for k, v in self.data_start_dates.items():
            print(f"{k}: {v}")
    
    def next(self):
        # 检查是否到调仓日（每3个月的第一个交易日）
        dt = self.data.datetime.date(0)
        
        # 首次运行或到达调仓月份
        if self.last_rebalance is None or (
            dt.month >= (self.last_rebalance.month + self.p.rebalance_months) % 12 
            and dt.year > self.last_rebalance.year
        ):
            self.rebalance_portfolio()
            self.last_rebalance = dt
    
    def rebalance_portfolio(self):
        """优化后的调仓逻辑"""
        dt = self.data.datetime.date(0)
        print(f"\n{dt} 开始调仓...")
        
        # 计算过去90个交易日的收益率（确保有足够数据）
        returns = {}
        valid_tickers = []
        
        for data in self.datas:
            ticker = data._name
            days_needed = 90
            
            # 检查数据是否足够
            available_days = len(data.close.array)
            if available_days < days_needed:
                print(f"跳过 {ticker} - 只有 {available_days} 天数据 (< {days_needed})")
                continue
                
            # 计算收益率
            start_idx = -days_needed
            start_price = data.close[start_idx]
            end_price = data.close[0]
            
            if start_price > 0:
                returns[ticker] = (end_price - start_price) / start_price
                valid_tickers.append(ticker)
        
        if len(returns) < self.p.top_n:
            print(f"可用ETF不足 {self.p.top_n} 个，实际有 {len(returns)} 个")
            return
            
        # 选择表现最好的4只ETF
        sorted_returns = sorted(returns.items(), key=lambda x: x[1], reverse=True)
        selected = sorted_returns[:self.p.top_n]
        
        # 计算目标市值
        total_value = self.broker.getvalue()
        target_per_etf = total_value * self.p.weight
        
        print(f"\n当前总资产: {total_value:.2f}")
        print("入选ETF及收益率:")
        for ticker, ret in selected:
            print(f"{ticker}: {ret*100:.2f}%")
        
        # 调整仓位
        for data in self.datas:
            ticker = data._name
            position = self.getposition(data)
            current_value = position.size * data.close[0] if position.size else 0
            
            # 不在入选列表中的清仓
            if ticker not in returns:
                if position.size > 0:
                    print(f"清仓 {ticker} ({position.size}股)")
                    self.close(data)
                continue
                
            # 入选ETF的仓位调整
            if ticker in dict(selected):
                target_size = int(target_per_etf / data.close[0])
                delta = target_size - position.size
                
                if delta > 0:
                    print(f"买入 {ticker}: {delta}股 @ {data.close[0]:.2f}")
                    self.buy(data=data, size=delta)
                elif delta < 0:
                    print(f"卖出 {ticker}: {-delta}股")
                    self.sell(data=data, size=-delta)
            else:
                if position.size > 0:
                    print(f"卖出落选 {ticker} ({position.size}股)")
                    self.close(data)
        
        # 记录绩效
        self.performance.append((dt, total_value))
        print(f"{dt} 调仓完成")


In [6]:
# %% [markdown]
# ## 6. 重新运行优化后的策略
# %%
# 创建新的回测引擎
cerebro_opt = bt.Cerebro()
cerebro_opt.broker.setcash(initial_cash)
cerebro_opt.broker.setcommission(commission=0.001)

# 添加数据
for ticker, data in datafeeds.items():
    cerebro_opt.adddata(data, name=ticker)

# 添加优化后的策略
cerebro_opt.addstrategy(OptimizedTop4Strategy)

# 添加相同的分析器
cerebro_opt.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro_opt.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.03)
cerebro_opt.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# 运行回测
print("开始优化后的回测...")
results_opt = cerebro_opt.run()
strat_opt = results_opt[0]

# 打印最终结果
print("\n最终资产:", strat_opt.broker.getvalue())

开始优化后的回测...
各ETF数据起始日期:
QQQ: 2025-06-30
SPY: 2025-06-30
159980: 2025-07-01
513010: 2025-07-01
159892: 2025-07-01
159934: 2025-07-01
159985: 2025-07-01
510880: 2025-07-01
516780: 2025-07-01

2021-10-19 开始调仓...

当前总资产: 100000.00
入选ETF及收益率:
159892: 84.01%
516780: 21.22%
513010: 10.12%
159980: -4.78%
买入 159980: 15673股 @ 1.59
买入 513010: 31055股 @ 0.81
买入 159892: 24975股 @ 1.00
买入 516780: 19113股 @ 1.31
2021-10-19 调仓完成

2022-01-03 开始调仓...

当前总资产: 90333.58
入选ETF及收益率:
159892: 30.36%
516780: 10.85%
QQQ: 7.43%
SPY: 7.13%
买入 QQQ: 57股 @ 392.64
买入 SPY: 49股 @ 454.47
卖出落选 159980 (15673股)
卖出落选 513010 (31055股)
买入 159892: 3611股 @ 0.79
买入 516780: 15792股 @ 1.43
2022-01-03 调仓完成

2023-04-03 开始调仓...

当前总资产: 75869.68
入选ETF及收益率:
QQQ: 14.02%
513010: 10.53%
159934: 6.69%
SPY: 5.02%
买入 QQQ: 3股 @ 315.74
买入 SPY: 47股 @ 398.81
买入 513010: 33452股 @ 0.57
卖出落选 159892 (28586股)
买入 159934: 4539股 @ 4.18
卖出落选 516780 (15792股)
2023-04-03 调仓完成

2024-07-01 开始调仓...

当前总资产: 92118.56
入选ETF及收益率:
159980: 14.62%
513010: 14.38%
159934: 13.

In [10]:
# %% [markdown]
# ## 7. 运行回测
# %%
print("开始回测...")
results = cerebro.run()
strat = results[0]

开始回测...

2021-10-19 开始调仓...
159892 数据不足90天
当前总资产: 100000.00
买入 QQQ: 68股 @ 366.55
买入 159980: 15673股 @ 1.59
买入 510880: 6180股 @ 4.04
买入 516780: 19113股 @ 1.31
2021-10-19 调仓完成
选中ETF: 516780, 159980, QQQ, 510880

2022-01-14 开始调仓...
159892 数据不足90天
当前总资产: 99012.28
卖出 510880: 6180股
调整 QQQ: 卖出1股
买入 SPY: 55股 @ 442.11
买入 159980: 644股 @ 1.52
买入 159934: 6872股 @ 3.60
2022-01-14 调仓完成
选中ETF: 159980, SPY, QQQ, 159934

2022-05-23 开始调仓...
当前总资产: 92926.08
卖出 QQQ: 67股
卖出 SPY: 55股
调整 159980: 卖出1733股
调整 159934: 卖出820股
买入 159985: 14054股 @ 1.65
买入 510880: 5953股 @ 3.90
2022-05-23 调仓完成
选中ETF: 159985, 159980, 159934, 510880

2022-09-26 开始调仓...
当前总资产: 90375.07
卖出 159980: 14584股
买入 159934: 52股 @ 3.70
调整 159985: 卖出1076股
买入 510880: 5695股 @ 3.97
买入 516780: 19888股 @ 1.14
2022-09-26 调仓完成
选中ETF: 159985, 516780, 510880, 159934

2023-01-31 开始调仓...
当前总资产: 95670.25
卖出 159934: 6104股
卖出 510880: 5695股
买入 SPY: 60股 @ 392.98
买入 159980: 14566股 @ 1.64
买入 159892: 32853股 @ 0.73
调整 159985: 卖出971股
2023-01-31 调仓完成
选中ETF: 159892, 159980, 1

In [11]:
# %% [markdown]
# ## 8. 分析结果
# %%
# 获取分析结果
returns_analyzer = strat.analyzers.returns.get_analysis()
sharpe_analyzer = strat.analyzers.sharpe.get_analysis()
drawdown_analyzer = strat.analyzers.drawdown.get_analysis()
trades_analyzer = strat.analyzers.trades.get_analysis()

# 打印绩效指标
print("\n绩效指标:")
print(f"初始资金: {initial_cash:.2f}")
print(f"最终资金: {strat.broker.getvalue():.2f}")
print(f"总收益率: {returns_analyzer['rtot']*100:.2f}%")
print(f"年化收益率: {returns_analyzer['rnorm100']:.2f}%")
print(f"夏普比率: {sharpe_analyzer['sharperatio']:.2f}")
print(f"最大回撤: {drawdown_analyzer['max']['drawdown']:.2f}%")
print(f"总交易次数: {trades_analyzer['total']['total']}")
print(f"盈利交易占比: {trades_analyzer['won']['total']/trades_analyzer['total']['total']*100:.1f}%")

# 打印季度收益率
if len(strat.performance) > 1:
    print("\n季度收益率:")
    for i in range(1, len(strat.performance)):
        date, value = strat.performance[i]
        prev_value = strat.performance[i-1][1]
        ret = (value - prev_value) / prev_value
        print(f"{date}: {ret*100:.2f}%")



绩效指标:
初始资金: 100000.00
最终资金: 105317.28
总收益率: 5.18%
年化收益率: 1.26%
夏普比率: -0.24
最大回撤: 18.38%
总交易次数: 18
盈利交易占比: 27.8%

季度收益率:
2022-01-14: -0.99%
2022-05-23: -6.15%
2022-09-26: -2.75%
2023-01-31: 5.86%
2023-06-06: -3.94%
2023-10-10: 1.43%
2024-02-14: -1.17%
2024-06-18: 9.83%
2024-10-23: 7.26%


In [12]:
# %% [markdown]
# ## 9. 可视化结果
# %%
# 绘制结果
cerebro.plot(style='candlestick', volume=False, figsize=(15, 10))

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 11 Axes>]]

In [7]:
# %% [markdown]
# ## 4. 修正后的季度轮动策略
# %%
class CorrectQuarterlyStrategy(bt.Strategy):
    params = (
        ('selection_count', 4),  # 每次选4只ETF
        ('weights', [0.25, 0.25, 0.25, 0.25]),  # 等权重分配
        ('quarter_months', [1, 4, 7, 10]),  # 季度调仓月份
    )

    def __init__(self):
        # 记录每个交易日
        self.dates = [self.data.datetime.date(0)]
        
        # 添加季度调仓标记
        self.quarter_ends = self.calculate_quarter_ends()
    
    def calculate_quarter_ends(self):
        """计算所有季度末日期"""
        dates = []
        current_date = start_date
        while current_date <= end_date:
            # 找每个季度的最后一天
            quarter_end = current_date + pd.offsets.QuarterEnd()
            if quarter_end > end_date:
                break
            dates.append(quarter_end.date())
            current_date = quarter_end + timedelta(days=1)
        return dates

    def next(self):
        current_date = self.data.datetime.date(0)
        self.dates.append(current_date)
        
        # 检查是否是季度末调仓日
        if current_date in self.quarter_ends:
            self.rebalance_portfolio(current_date)

    def rebalance_portfolio(self, current_date):
        """季度调仓逻辑"""
        print(f"\n{current_date} 开始季度调仓...")
        
        # 计算过去一个季度的收益率
        returns = {}
        valid_data = []
        
        for data in self.datas:
            try:
                # 获取本季度数据
                lookback = 63  # 约3个月的交易日
                if len(data) < lookback:
                    continue
                
                start_price = data.close[-lookback]
                end_price = data.close[0]
                returns[data._name] = (end_price - start_price) / start_price
                valid_data.append(data)
            except Exception as e:
                print(f"计算 {data._name} 收益率时出错: {str(e)}")
                continue
        
        if len(returns) < self.p.selection_count:
            print(f"可用ETF不足 {self.p.selection_count} 个，实际有 {len(returns)} 个")
            return
        
        # 按收益率排序
        sorted_returns = sorted(returns.items(), key=lambda x: x[1], reverse=True)
        selected = sorted_returns[:self.p.selection_count]
        print(f"本季度入选ETF: {[x[0] for x in selected]}")
        
        # 计算总资产
        total_value = self.broker.getvalue()
        print(f"调仓前总资产: {total_value:.2f}")
        
        # 先清空所有持仓
        for data in self.datas:
            if self.getposition(data).size > 0:
                self.close(data)
        
        # 按权重买入新选中的ETF
        for i, (ticker, _) in enumerate(selected):
            data = [d for d in self.datas if d._name == ticker][0]
            target_value = total_value * self.p.weights[i]
            shares = int(target_value / data.close[0])
            self.buy(data=data, size=shares)
            print(f"买入 {ticker}: {shares}股 @ {data.close[0]:.2f}")
        
        # 记录调仓信息
        new_value = self.broker.getvalue()
        print(f"调仓后总资产: {new_value:.2f}")
        print(f"{current_date} 调仓完成")

# %% [markdown]
# ## 5. 运行修正后的回测
# %%
# 创建回测引擎
cerebro = bt.Cerebro()
cerebro.broker.setcash(initial_cash)
cerebro.broker.setcommission(commission=0.001)

# 添加数据
for ticker, data in datafeeds.items():
    cerebro.adddata(data, name=ticker)

# 添加修正后的策略
cerebro.addstrategy(CorrectQuarterlyStrategy)

# 添加分析器
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.03)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')

# 运行回测
print(f"开始回测 {start_date.date()} 至 {end_date.date()}...")
results = cerebro.run()

# %% [markdown]
# ## 6. 分析结果
# %%
strat = results[0]

# 打印季度调仓日期
print("\n季度调仓日期:")
quarter_dates = [d for d in strat.dates if d in strat.quarter_ends]
for i, date in enumerate(quarter_dates, 1):
    print(f"第{i}次调仓: {date}")

# 绩效分析
returns = strat.analyzers.returns.get_analysis()
sharpe = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()

print("\n绩效指标:")
print(f"初始资金: {initial_cash:.2f}")
print(f"最终资金: {strat.broker.getvalue():.2f}")
print(f"总收益率: {returns['rtot']*100:.2f}%")
print(f"年化收益率: {returns['rnorm100']:.2f}%")
print(f"夏普比率: {sharpe['sharperatio']:.2f}")
print(f"最大回撤: {drawdown['max']['drawdown']:.2f}%")

开始回测 2021-07-01 至 2025-07-01...

2021-12-31 开始季度调仓...
本季度入选ETF: ['QQQ', 'SPY', '516780', '159980']
调仓前总资产: 100000.00
买入 QQQ: 64股 @ 388.89
买入 SPY: 55股 @ 451.85
买入 516780: 17482股 @ 1.43
买入 159980: 16756股 @ 1.49
调仓后总资产: 100000.00
2021-12-31 调仓完成

2022-03-31 开始季度调仓...
本季度入选ETF: ['159985', '159980', '510880', '159934']
调仓前总资产: 95150.14
买入 159985: 14960股 @ 1.59
买入 159980: 13927股 @ 1.71
买入 510880: 5896股 @ 4.03
买入 159934: 6264股 @ 3.80
调仓后总资产: 95150.14
2022-03-31 调仓完成

2022-06-30 开始季度调仓...
本季度入选ETF: ['513010', '516780', '159892', '510880']
调仓前总资产: 90728.12
买入 513010: 36291股 @ 0.62
买入 516780: 17434股 @ 1.30
买入 159892: 33652股 @ 0.67
买入 510880: 5690股 @ 3.99
调仓后总资产: 90728.12
2022-06-30 调仓完成

2022-09-30 开始季度调仓...
本季度入选ETF: ['159985', '159980', '510880', '159934']
调仓前总资产: 75458.86
买入 159985: 10633股 @ 1.77
买入 159980: 13379股 @ 1.41
买入 510880: 4737股 @ 3.98
买入 159934: 5037股 @ 3.75
调仓后总资产: 75458.86
2022-09-30 调仓完成

2023-03-31 开始季度调仓...
本季度入选ETF: ['QQQ', '159934', 'SPY', '510880']
调仓前总资产: 77952.03
买入 QQQ: 6

In [9]:
# %% [markdown]
# ## 1. 数据准备修正
# %%
import pandas as pd
import numpy as np
import backtrader as bt
from datetime import datetime

def get_quarter_end_dates(start_date, end_date):
    """生成精确的季度末日期列表"""
    quarters = pd.date_range(start_date, end_date, freq='Q')
    return [date.date() for date in quarters]

# 测试日期生成
start_date = datetime(2021, 7, 1)
end_date = datetime(2025, 7, 1)
quarter_ends = get_quarter_end_dates(start_date, end_date)
print("所有季度末日期:", quarter_ends)

# %% [markdown]
# ## 2. 修正收益率计算
# %%
class FixedQuarterlyStrategy(bt.Strategy):
    params = (
        ('top_n', 4),
        ('weight', 0.25),
    )

    def __init__(self):
        self.quarter_ends = get_quarter_end_dates(
            self.data.datetime.date(0), 
            self.params.to_date
        )
        self.quarter_count = 0
        
    def next(self):
        current_date = self.data.datetime.date(0)
        
        # 精确匹配季度末调仓
        if current_date in self.quarter_ends:
            self.quarter_count += 1
            self.rebalance(current_date)
    
    def rebalance(self, current_date):
        print(f"\n=== 第{self.quarter_count}次调仓 {current_date} ===")
        
        # 计算过去90个交易日的收益率（确保足够数据）
        returns = {}
        for data in self.datas:
            try:
                # 获取90个交易日前的索引
                lookback = min(90, len(data))
                if lookback < 60:  # 至少需要60个交易日数据
                    continue
                    
                start_price = data.close[-lookback]
                end_price = data.close[0]
                if start_price > 0:
                    returns[data._name] = (end_price - start_price) / start_price
            except Exception as e:
                print(f"计算 {data._name} 收益率出错:", str(e))
        
        if len(returns) < self.p.top_n:
            print(f"可用ETF不足 {self.p.top_n} 个，实际 {len(returns)} 个")
            return
            
        # 选择收益率最高的前4名
        selected = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:self.p.top_n]
        print("入选ETF及收益率:")
        for ticker, ret in selected:
            print(f"{ticker}: {ret*100:.2f}%")
        
        # 计算总资产
        total_value = self.broker.getvalue()
        print(f"调仓前总资产: {total_value:.2f}")
        
        # 清空所有现有持仓
        for data in self.datas:
            if self.getposition(data).size > 0:
                self.close(data)
        
        # 等权重买入新选中的ETF
        for ticker, _ in selected:
            data = next(d for d in self.datas if d._name == ticker)
            target_value = total_value * self.p.weight
            shares = int(target_value / data.close[0])
            self.buy(data=data, size=shares)
            print(f"买入 {ticker}: {shares}股 @ {data.close[0]:.2f}")
        
        print(f"调仓后总资产: {self.broker.getvalue():.2f}")

# %% [markdown]
# ## 3. 正确运行回测
# %%
# 重新准备数据（确保日期范围正确）
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(0.001)

# 添加数据（示例代码，需替换为实际数据加载逻辑）
for ticker in ['QQQ', 'SPY', '510880', '159934']:
    data = bt.feeds.PandasData(
        dataname=yf.download(ticker, start='2021-07-01', end='2025-07-01'),
        fromdate=datetime(2021,7,1),
        todate=datetime(2025,7,1)
    )
    cerebro.adddata(data, name=ticker)

# 添加策略
cerebro.addstrategy(
    FixedQuarterlyStrategy,
    to_date=datetime(2025,7,1).date()
)

# 添加分析器
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.03)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# 运行回测
print("开始回测...")
results = cerebro.run()
strat = results[0]

# %% [markdown]
# ## 4. 验证结果
# %%
# 检查季度调仓次数
expected_quarters = len(get_quarter_end_dates(
    datetime(2021,7,1).date(),
    datetime(2025,7,1).date()
))
print(f"应执行调仓次数: {expected_quarters}")
print(f"实际调仓次数: {strat.quarter_count}")

# 绩效分析
returns = strat.analyzers.returns.get_analysis()
sharpe = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()

print("\n最终绩效:")
print(f"初始资金: 100000.00")
print(f"最终资金: {strat.broker.getvalue():.2f}")
print(f"总收益率: {returns['rtot']*100:.2f}%")
print(f"年化收益率: {returns['rnorm100']:.2f}%")
print(f"夏普比率: {sharpe['sharperatio']:.2f}")
print(f"最大回撤: {drawdown['max']['drawdown']:.2f}%")

# 绘制结果
cerebro.plot()

所有季度末日期: [datetime.date(2021, 9, 30), datetime.date(2021, 12, 31), datetime.date(2022, 3, 31), datetime.date(2022, 6, 30), datetime.date(2022, 9, 30), datetime.date(2022, 12, 31), datetime.date(2023, 3, 31), datetime.date(2023, 6, 30), datetime.date(2023, 9, 30), datetime.date(2023, 12, 31), datetime.date(2024, 3, 31), datetime.date(2024, 6, 30), datetime.date(2024, 9, 30), datetime.date(2024, 12, 31), datetime.date(2025, 3, 31), datetime.date(2025, 6, 30)]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['159934']: HTTPError('HTTP Error 404: ')
Failed to get ticker '159934' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['510880']: YFTzMissingError('possibly delisted; no timezone found')


开始回测...


AttributeError: 'tuple' object has no attribute 'lower'

In [19]:
# %% [markdown]
# ## 1. 导入库和初始化设置
# %%
import backtrader as bt
from backtrader import indicators  # 正确导入指标模块
import pandas as pd
import numpy as np
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置回测参数
start_date = datetime(2021, 7, 1)
end_date = datetime(2025, 7, 1)
initial_cash = 100000

# %% [markdown]
# ## 2. 定义ETF列表（国际+国内）
# %%
intl_etfs = {
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'
}

china_etfs = {
    '159980': '有色金属ETF',
    '513010': '恒生科技',
    '159892': '恒生医药',
    '159934': '黄金ETF',
    '159985': '豆粕ETF',
    '510880': '红利ETF',
    '516780': '稀土ETF'
}

all_tickers = list(intl_etfs.keys()) + list(china_etfs.keys())

# %% [markdown]
# ## 3. 数据获取函数
# %%
def prepare_data(tickers, start_date, end_date):
    """准备Backtrader兼容的数据"""
    datafeeds = {}
    
    for ticker in tickers:
        print(f"正在处理 {ticker}...", end=" ")
        try:
            if ticker in intl_etfs:
                # 国际ETF使用yfinance
                df = yf.download(ticker, start=start_date, end=end_date)
                if df.empty:
                    print("无数据")
                    continue
                df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
                df.columns = ['open', 'high', 'low', 'close', 'volume']
            else:
                # 国内ETF使用akshare
                df = ak.fund_etf_hist_em(
                    symbol=ticker,
                    period="daily",
                    start_date=start_date.strftime("%Y%m%d"),
                    end_date=end_date.strftime("%Y%m%d"),
                    adjust="hfq"
                )
                if df.empty:
                    print("无数据")
                    continue
                df = df.rename(columns={
                    '日期': 'date',
                    '开盘': 'open',
                    '最高': 'high',
                    '最低': 'low',
                    '收盘': 'close',
                    '成交量': 'volume'
                })
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
            
            # 转换为Backtrader数据格式
            data = bt.feeds.PandasData(
                dataname=df,
                fromdate=start_date,
                todate=end_date
            )
            datafeeds[ticker] = data
            print(f"已加载 {len(df)} 天数据")
        except Exception as e:
            print(f"失败: {str(e)}")
    
    return datafeeds

# %% [markdown]
# ## 4. 准备数据
# %%
print("开始获取数据...")
datafeeds = prepare_data(all_tickers, start_date, end_date)
print(f"\n成功获取 {len(datafeeds)}/{len(all_tickers)} 只ETF数据")

# %% [markdown]
# ## 5. 最终修正的策略类
# %%
class FinalQuarterlyStrategy(bt.Strategy):
    params = (
        ('top_n', 4),          # 选择前4名
        ('weight', 0.25),      # 每只ETF权重25%
        ('lookback_period', 63), # 约3个月的交易日
    )

    def __init__(self):
        # 存储每个数据的收盘价引用
        self.dclose = {data._name: data.close for data in self.datas}
        
        # 计算所有季度末日期（基于交易日历）
        self.quarter_ends = self._get_quarter_end_dates()
        self.quarter_count = 0
        self.performance = []
        
        print(f"生成的季度调仓日期: {self.quarter_ends}")

    def _get_quarter_end_dates(self):
        """获取实际的季度末交易日"""
        calendar = pd.bdate_range(
            start=self.data.datetime.date(0),
            end=self.data.datetime.date(len(self.data)-1)
        )
        quarter_ends = []
        for year in range(2021, 2026):
            for month in [3, 6, 9, 12]:
                month_end = calendar[calendar.year == year][calendar.month == month]
                if len(month_end) > 0:
                    quarter_ends.append(month_end[-1].date())
        return quarter_ends

    def next(self):
        current_date = self.data.datetime.date(0)
        
        # 调试输出当前日期
        if len(self) % 20 == 0:
            print(f"当前回测日期: {current_date}")
        
        # 精确季度末调仓
        if current_date in self.quarter_ends:
            self.quarter_count += 1
            self._rebalance(current_date)

    def _rebalance(self, current_date):
        """执行季度调仓（修正边界条件）"""
        print(f"\n=== 第 {self.quarter_count} 次调仓 {current_date} ===")
        
        # 初始化阶段处理（前3个月持有现金）
        if self.quarter_count == 1 and len(self) < 63:  # 约3个月交易日
            print("初始化阶段，持有现金")
            return
        
        # 计算过去一个季度的收益率
        returns = {}
        valid_etfs = []
        
        for data in self.datas:
            ticker = data._name
            try:
                # 动态确定回看周期
                lookback = min(63, len(data))  # 最多回看63个交易日（约3个月）
                
                # 最后季度特殊处理：使用所有可用数据
                if current_date >= self.quarter_ends[-1] - pd.Timedelta(days=7):  # 最后一周
                    lookback = len(data)
                    print(f"{ticker} 使用全部 {lookback} 天数据计算")
                
                # 最小数据要求（首季度降低要求）
                min_days = 20 if self.quarter_count <= 2 else 40
                if lookback < min_days:
                    print(f"跳过 {ticker} (只有 {lookback} 天数据)")
                    continue
                    
                # 获取价格数据
                prices = data.close.get(size=lookback)
                start_price = prices[0]
                end_price = prices[-1]
                
                # 价格有效性检查
                if start_price <= 0 or end_price <= 0:
                    print(f"跳过 {ticker} (无效价格)")
                    continue
                    
                # 计算收益率
                returns[ticker] = (end_price - start_price) / start_price
                valid_etfs.append(ticker)
                
            except Exception as e:
                print(f"计算 {ticker} 收益率出错: {str(e)}")
                continue
        
        # 可用ETF不足时的处理
        if len(returns) < self.p.num_etfs:
            if len(returns) > 0:
                print(f"可用ETF不足 {self.p.num_etfs} 个，实际 {len(returns)} 个，按实际数量配置")
                selected_num = len(returns)
                selected_weights = [1.0/selected_num] * selected_num  # 等权重调整
            else:
                print("无有效ETF可选，保持现金")
                return
        else:
            selected_num = self.p.num_etfs
            selected_weights = self.p.weights
        
        # 反向选择：收益率最低的ETF
        selected = sorted(returns.items(), key=lambda x: x[1])[:selected_num]
        print("入选ETF及收益率:")
        for ticker, ret in selected:
            print(f"{ticker}: {ret*100:.2f}%")
        
        # 计算总资产
        total_value = self.broker.getvalue()
        print(f"调仓前总资产: {total_value:.2f}")
        
        # 清空所有现有持仓
        for data in self.datas:
            if self.getposition(data).size > 0:
                self.close(data)
        
        # 按权重买入新选中的ETF
        for i, (ticker, _) in enumerate(selected):
            data = next(d for d in self.datas if d._name == ticker)
            target_value = total_value * selected_weights[i]
            shares = int(target_value / data.close[0])
            if shares > 0:
                self.buy(data=data, size=shares)
                print(f"买入 {ticker}: {shares}股 @ {data.close[0]:.2f}")
            else:
                print(f"{ticker} 目标股数为0，跳过")
        
        # 记录绩效
        new_value = self.broker.getvalue()
        self.trade_log.append((current_date, new_value))
        print(f"调仓后总资产: {new_value:.2f}")

# %% [markdown]
# ## 6. 回测引擎设置
# %%
# 创建回测引擎
cerebro = bt.Cerebro()
cerebro.broker.setcash(initial_cash)
cerebro.broker.setcommission(commission=0.001)  # 0.1%交易佣金

# 添加数据
for ticker, data in datafeeds.items():
    cerebro.adddata(data, name=ticker)

# 添加策略
cerebro.addstrategy(FinalQuarterlyStrategy)

# 添加分析器
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.03)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

# %% [markdown]
# ## 7. 运行回测
# %%
print("开始回测...")
results = cerebro.run()
strat = results[0]

# %% [markdown]
# ## 8. 改进的结果分析
# %%
# 检查季度调仓次数
expected_quarters = len(pd.date_range(start_date, end_date, freq='Q'))
print(f"\n应执行调仓次数: {expected_quarters}")
print(f"实际调仓次数: {strat.quarter_count}")

# 获取分析结果
returns = strat.analyzers.returns.get_analysis()
sharpe = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
trades = strat.analyzers.trades.get_analysis()

# 安全获取绩效指标
def safe_get(dct, keys, default=None):
    for key in keys:
        try:
            dct = dct[key]
        except (KeyError, TypeError):
            return default
    return dct

# 打印绩效指标
print("\n=== 最终绩效 ===")
print(f"初始资金: {initial_cash:,.2f}")
print(f"最终资金: {strat.broker.getvalue():,.2f}")

# 处理可能为None的指标
total_return = safe_get(returns, ['rtot'], 0)
annual_return = safe_get(returns, ['rnorm100'], 0)
sharpe_ratio = safe_get(sharpe, ['sharperatio'], 0)
max_drawdown = safe_get(drawdown, ['max', 'drawdown'], 0)
total_trades = safe_get(trades, ['total', 'total'], 0)
won_trades = safe_get(trades, ['won', 'total'], 0)

print(f"总收益率: {total_return*100:.2f}%")
print(f"年化收益率: {annual_return:.2f}%")
print(f"夏普比率: {sharpe_ratio:.2f}" if sharpe_ratio is not None else "夏普比率: 无数据")
print(f"最大回撤: {max_drawdown*100:.2f}%")
print(f"总交易次数: {total_trades}")
print(f"盈利交易占比: {won_trades/total_trades*100:.1f}%" if total_trades > 0 else "盈利交易占比: 无交易")

# 打印季度收益率
if len(strat.performance) > 1:
    print("\n=== 季度收益率 ===")
    for i in range(1, len(strat.performance)):
        date, value = strat.performance[i]
        prev_value = strat.performance[i-1][1]
        if prev_value != 0:
            ret = (value - prev_value) / prev_value
            print(f"{date}: {ret*100:.2f}%")
        else:
            print(f"{date}: 零值无法计算收益率")

# %% [markdown]
# ## 9. 可视化结果
# %%
# 绘制结果
cerebro.plot(style='candlestick', volume=False, figsize=(15, 10))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

开始获取数据...
正在处理 QQQ... 已加载 1003 天数据
正在处理 SPY... 已加载 1003 天数据
正在处理 159980... 

已加载 969 天数据
正在处理 513010... 已加载 969 天数据
正在处理 159892... 已加载 898 天数据
正在处理 159934... 已加载 969 天数据
正在处理 159985... 已加载 969 天数据
正在处理 510880... 已加载 969 天数据
正在处理 516780... 已加载 969 天数据

成功获取 9/9 只ETF数据
开始回测...
生成的季度调仓日期: []
当前回测日期: 2021-10-20
当前回测日期: 2021-11-17
当前回测日期: 2021-12-15
当前回测日期: 2022-01-12
当前回测日期: 2022-02-09
当前回测日期: 2022-03-09
当前回测日期: 2022-04-06
当前回测日期: 2022-05-04
当前回测日期: 2022-06-01
当前回测日期: 2022-06-29
当前回测日期: 2022-07-27
当前回测日期: 2022-08-24
当前回测日期: 2022-09-21
当前回测日期: 2022-10-19
当前回测日期: 2022-11-16
当前回测日期: 2022-12-14
当前回测日期: 2023-01-12
当前回测日期: 2023-02-09
当前回测日期: 2023-03-09
当前回测日期: 2023-04-06
当前回测日期: 2023-05-04
当前回测日期: 2023-06-01
当前回测日期: 2023-06-29
当前回测日期: 2023-07-27
当前回测日期: 2023-08-24
当前回测日期: 2023-09-21
当前回测日期: 2023-10-19
当前回测日期: 2023-11-16
当前回测日期: 2023-12-14
当前回测日期: 2024-01-12
当前回测日期: 2024-02-09
当前回测日期: 2024-03-08
当前回测日期: 2024-04-05
当前回测日期: 2024-05-03
当前回测日期: 2024-05-31
当前回测日期: 2024-06-28
当前回测日期: 2024-07-26
当前回测日期: 2024-08-23
当前回测日期: 2024-09-20
当前回测日期: 2024-10-18
当前回测日期: 2024-11-15
当前回测日期: 

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 11 Axes>]]

In [22]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import numpy as np
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 数据获取函数 - 与原始代码保持一致
def fetch_etf_data_ak(symbol, start_date, end_date):
    """通过AKShare获取ETF数据"""
    try:
        adjusted_start_date = start_date + timedelta(days=7)
        df = ak.fund_etf_hist_em(symbol=symbol, period="daily", 
                                start_date=adjusted_start_date.strftime("%Y%m%d"), 
                                end_date=end_date.strftime("%Y%m%d"), 
                                adjust="hfq")
        df = df.rename(columns={'日期':'date', '收盘':'close'})
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        return df[['close']]
    except Exception as e:
        print(f"获取{symbol}数据失败: {e}")
        return None

# 设置回测日期范围
end_date = datetime(2025, 7, 22)
start_date = end_date - timedelta(days=7*365)

# ================== 数据准备部分 ==================
class DataPreparation:
    def __init__(self):
        self.intl_etfs = {
            'QQQ': '纳斯达克ETF',
            'SPY': '标普500ETF'
        }
        
        self.china_etfs = {
            '159980': '有色金属etf',
            '513010': '恒生科技',
            '159892': '恒生医药',
            '159934': '黄金etf',
            '159985': '豆粕etf',
            '510880': '红利ETF',
            '516780': '稀土'
        }
    
    def prepare_data(self):
        cerebro = bt.Cerebro()
        all_dates = pd.date_range(start_date, end_date)
        
        # 获取国际ETF数据
        for ticker, name in self.intl_etfs.items():
            print(f"获取{ticker}数据...")
            adjusted_start_date = start_date + timedelta(days=7)
            df = yf.download(ticker, start=adjusted_start_date, end=end_date)
            
            if not df.empty:
                # 确保列名符合Backtrader要求
                df = df.rename(columns={
                    'Open': 'open',
                    'High': 'high',
                    'Low': 'low',
                    'Close': 'close',
                    'Volume': 'volume'
                })
                
                # 重新索引并填充缺失值
                df = df.reindex(all_dates).fillna(method='ffill')
                
                # 确保只包含所需列
                df = df[['open', 'high', 'low', 'close', 'volume']]
                
                # 创建Backtrader数据源
                data = bt.feeds.PandasData(
                    dataname=df,
                    fromdate=start_date,
                    todate=end_date
                )
                cerebro.adddata(data, name=ticker)
        
        # 获取中国ETF数据
        for code, name in self.china_etfs.items():
            print(f"获取{name}({code})数据...")
            df = fetch_etf_data_ak(code, start_date, end_date)
            
            if df is not None and not df.empty:
                # 创建完整的OHLCV数据
                df['open'] = df['close']
                df['high'] = df['close']
                df['low'] = df['close']
                df['volume'] = 0
                
                # 重新索引并填充缺失值
                df = df.reindex(all_dates).fillna(method='ffill')
                
                # 确保只包含所需列
                df = df[['open', 'high', 'low', 'close', 'volume']]
                
                # 创建Backtrader数据源
                data = bt.feeds.PandasData(
                    dataname=df,
                    fromdate=start_date,
                    todate=end_date
                )
                cerebro.adddata(data, name=code)
        
        # 添加标普500作为基准
        print("获取标普500指数数据...")
        sp500 = yf.download('^GSPC', start=start_date + timedelta(days=7), end=end_date)
        
        if not sp500.empty:
            # 确保列名符合Backtrader要求
            sp500 = sp500.rename(columns={
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume'
            })
            
            # 重新索引并填充缺失值
            sp500 = sp500.reindex(all_dates).fillna(method='ffill')
            
            # 确保只包含所需列
            sp500 = sp500[['open', 'high', 'low', 'close', 'volume']]
            
            # 创建Backtrader数据源
            sp500_data = bt.feeds.PandasData(
                dataname=sp500,
                fromdate=start_date,
                todate=end_date
            )
            cerebro.adddata(sp500_data, name='^GSPC')
        
        return cerebro

# ================== 策略部分 ==================
class QuarterlyReverseRotation(bt.Strategy):
    params = (
        ('num_assets', 4),  # 每季度选择的ETF数量
        ('initial_cash', 100000),  # 初始资金
    )
    
    def __init__(self):
        # 保存所有数据引用
        self.datas_dict = {data._name: data for data in self.datas}
        self.quarterly_rebalance_dates = []
        
        # 计算季度末日期
        self.quarters = pd.date_range(start_date, end_date, freq='Q')
        if end_date not in self.quarters:
            self.quarters = self.quarters.append(pd.DatetimeIndex([end_date]))
        
        # 当前持有的ETF
        self.current_holdings = set()
        
        # 记录季度选择和表现
        self.quarterly_selections = []
        self.quarterly_performance = []
        
        # 记录组合价值
        self.portfolio_values = []
        self.dates = []
        
        # 添加分析器
        self.add_analyzer()
    
    def add_analyzer(self):
        self.analyzers.returns = btanalyzers.Returns()
        self.analyzers.sharpe = btanalyzers.SharpeRatio(riskfreerate=0.03)
        self.analyzers.drawdown = btanalyzers.DrawDown()
        self.analyzers.volatility = btanalyzers.AnnualizedVolatility()
        self.analyzers.tradeanalyzer = btanalyzers.TradeAnalyzer()
    
    def next(self):
        # 记录每日组合价值
        self.dates.append(self.data.datetime.date())
        total_value = self.broker.getvalue()
        self.portfolio_values.append(total_value)
        
        # 检查是否是季度再平衡日
        current_date = self.data.datetime.date()
        if current_date in [q.date() for q in self.quarters]:
            self.rebalance_portfolio(current_date)
    
    def rebalance_portfolio(self, current_date):
        # 计算过去一个季度的收益率
        prev_quarter_start = current_date - pd.offsets.QuarterEnd()
        returns = {}
        
        for name, data in self.datas_dict.items():
            if name == '^GSPC':  # 跳过基准
                continue
                
            try:
                # 获取当前和上一季度初的价格
                idx = self.dates.index(current_date)
                prev_idx = self.dates.index(prev_quarter_start) if prev_quarter_start in self.dates else 0
                
                current_price = data.close[0]
                prev_price = data.close[-idx+prev_idx] if idx > prev_idx else data.close[0]
                
                if current_price > 0 and prev_price > 0:
                    returns[name] = (current_price - prev_price) / prev_price
            except (ValueError, IndexError):
                continue
        
        # 选择收益率最低的ETF
        if len(returns) >= self.p.num_assets:
            selected = sorted(returns.items(), key=lambda x: x[1])[:self.p.num_assets]
            selected_tickers = [x[0] for x in selected]
            selected_names = selected_tickers  # 简化处理，实际可以使用映射
            
            # 记录季度选择
            self.quarterly_selections.append((current_date, selected_tickers, selected_names))
        else:
            selected_tickers = []
            self.quarterly_selections.append((current_date, [], ["现金"]))
        
        # 卖出不在新选择中的持仓
        for name in list(self.current_holdings):
            if name not in selected_tickers:
                self.close(data=self.datas_dict[name])
                self.current_holdings.remove(name)
        
        # 计算当前总价值
        total_value = self.broker.getvalue()
        
        # 买入新增的ETF
        if selected_tickers:
            allocation = total_value / len(selected_tickers)
            
            # 调整现有持仓
            for name in set(self.current_holdings) & set(selected_tickers):
                data = self.datas_dict[name]
                current_value = self.getposition(data).size * data.close[0]
                delta = allocation - current_value
                
                if delta > 0:  # 需要买入更多
                    self.order_target_value(data=data, target=allocation)
                elif delta < 0:  # 需要卖出部分
                    self.order_target_value(data=data, target=allocation)
            
            # 买入新增ETF
            for name in set(selected_tickers) - set(self.current_holdings):
                data = self.datas_dict[name]
                self.order_target_value(data=data, target=allocation)
                self.current_holdings.add(name)
        
        # 记录季度表现
        if len(self.quarterly_performance) > 0:
            last_quarter_value = self.quarterly_performance[-1][1]
            quarter_return = (total_value - last_quarter_value) / last_quarter_value
        else:
            quarter_return = 0
        
        self.quarterly_performance.append((current_date, total_value, quarter_return))
    
    def stop(self):
        # 回测结束时打印结果
        print("\n季度选择记录及收益率:")
        for i, quarter in enumerate(self.quarterly_selections):
            date = quarter[0].strftime('%Y-%m-%d')
            tickers = ", ".join(quarter[1]) if quarter[1] else "现金"
            names = ", ".join(quarter[2]) if quarter[2] else "现金"
            q_return = self.quarterly_performance[i][2] * 100
            print(f"{date}: 选择基金 {tickers} ({names}) | 季度收益率: {q_return:.2f}%")
        
        # 打印分析结果
        self.print_analyzers()
    
    def print_analyzers(self):
        print("\n策略绩效指标:")
        print(f"初始资金: {self.p.initial_cash:,.2f}元")
        print(f"最终资金: {self.broker.getvalue():,.2f}元")
        
        # 总收益率
        rets = self.analyzers.returns.get_analysis()
        print(f"总收益率: {rets['rtot']*100:.2f}%")
        print(f"年化收益率: {rets['rnorm']*100:.2f}%")
        
        # 夏普比率
        sharpe = self.analyzers.sharpe.get_analysis()
        print(f"夏普比率: {sharpe['sharperatio']:.2f}")
        
        # 最大回撤
        drawdown = self.analyzers.drawdown.get_analysis()
        print(f"最大回撤: {drawdown['max']['drawdown']*100:.2f}%")
        
        # 年化波动率
        vol = self.analyzers.volatility.get_analysis()
        print(f"年化波动率: {vol['volatility']*100:.2f}%")

# ================== 回测执行部分 ==================
def run_backtest():
    # 准备数据
    data_prep = DataPreparation()
    cerebro = data_prep.prepare_data()
    
    # 添加策略
    cerebro.addstrategy(QuarterlyReverseRotation)
    
    # 设置初始资金
    cerebro.broker.setcash(100000)
    
    # 设置佣金
    cerebro.broker.setcommission(commission=0.001)  # 0.1%佣金
    
    # 添加分析器
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', riskfreerate=0.03)
    cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    
    # 运行回测
    print('初始资金: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    print('最终资金: %.2f' % cerebro.broker.getvalue())
    
    # 绘制结果
    cerebro.plot(style='candlestick')

if __name__ == '__main__':
    run_backtest()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

获取QQQ数据...
获取SPY数据...
获取有色金属etf(159980)数据...


获取恒生科技(513010)数据...
获取恒生医药(159892)数据...
获取黄金etf(159934)数据...
获取豆粕etf(159985)数据...
获取红利ETF(510880)数据...
获取稀土(516780)数据...


[*********************100%***********************]  1 of 1 completed

获取标普500指数数据...
初始资金: 100000.00


AttributeError: 'tuple' object has no attribute 'lower'

In [23]:
import backtrader as bt
import pandas as pd
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta

# 设置回测日期范围
end_date = datetime(2025, 7, 22)
start_date = end_date - timedelta(days=7*365)

# 数据获取函数 - 修正版
def fetch_etf_data_ak(symbol, start_date, end_date):
    """通过AKShare获取ETF数据并转换为标准格式"""
    try:
        df = ak.fund_etf_hist_em(
            symbol=symbol, 
            period="daily", 
            start_date=start_date.strftime("%Y%m%d"), 
            end_date=end_date.strftime("%Y%m%d"), 
            adjust="hfq"
        )
        # 转换为标准OHLCV格式
        df = df.rename(columns={'日期': 'date', '收盘': 'close'})
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        df['open'] = df['close']
        df['high'] = df['close']
        df['low'] = df['close']
        df['volume'] = 0  # AKShare不提供成交量数据
        return df[['open', 'high', 'low', 'close', 'volume']]
    except Exception as e:
        print(f"获取{symbol}数据失败: {e}")
        return None

# 数据准备类 - 完全重构
class DataPreparation:
    def __init__(self):
        self.etf_list = {
            'yfinance': {
                'QQQ': '纳斯达克ETF',
                'SPY': '标普500ETF',
                '^GSPC': '标普500指数'  # 作为基准
            },
            'akshare': {
                '159980': '有色金属ETF',
                '513010': '恒生科技ETF',
                '159892': '恒生医药ETF'
            }
        }
    
    def prepare_data(self):
        cerebro = bt.Cerebro()
        all_dates = pd.date_range(start_date, end_date, freq='D')
        
        # 处理yfinance数据
        for ticker, name in self.etf_list['yfinance'].items():
            print(f"正在获取 {name}({ticker}) 数据...")
            try:
                df = yf.download(ticker, start=start_date, end=end_date)
                if df.empty:
                    continue
                    
                # 标准化列名和格式
                df = df.rename(columns={
                    'Open': 'open',
                    'High': 'high',
                    'Low': 'low',
                    'Close': 'close',
                    'Volume': 'volume'
                })
                
                # 确保日期连续
                df = df.reindex(all_dates).fillna(method='ffill')
                
                # 添加到Cerebro
                data = bt.feeds.PandasData(
                    dataname=df[['open', 'high', 'low', 'close', 'volume']],
                    fromdate=start_date,
                    todate=end_date
                )
                cerebro.adddata(data, name=ticker)
                
            except Exception as e:
                print(f"处理 {ticker} 数据时出错: {e}")
        
        # 处理AKShare数据
        for code, name in self.etf_list['akshare'].items():
            print(f"正在获取 {name}({code}) 数据...")
            try:
                df = fetch_etf_data_ak(code, start_date, end_date)
                if df is None or df.empty:
                    continue
                    
                # 确保日期连续
                df = df.reindex(all_dates).fillna(method='ffill')
                
                # 添加到Cerebro
                data = bt.feeds.PandasData(
                    dataname=df,
                    fromdate=start_date,
                    todate=end_date
                )
                cerebro.adddata(data, name=code)
                
            except Exception as e:
                print(f"处理 {code} 数据时出错: {e}")
        
        return cerebro

# 策略类保持不变
class QuarterlyReverseRotation(bt.Strategy):
    params = (
        ('num_assets', 4),  # 每季度选择的ETF数量
        ('initial_cash', 100000),  # 初始资金
    )
    
    def __init__(self):
        # 保存所有数据引用
        self.datas_dict = {data._name: data for data in self.datas}
        self.quarterly_rebalance_dates = []
        
        # 计算季度末日期
        self.quarters = pd.date_range(start_date, end_date, freq='Q')
        if end_date not in self.quarters:
            self.quarters = self.quarters.append(pd.DatetimeIndex([end_date]))
        
        # 当前持有的ETF
        self.current_holdings = set()
        
        # 记录季度选择和表现
        self.quarterly_selections = []
        self.quarterly_performance = []
        
        # 记录组合价值
        self.portfolio_values = []
        self.dates = []
        
        # 添加分析器
        self.add_analyzer()
    
    def add_analyzer(self):
        self.analyzers.returns = btanalyzers.Returns()
        self.analyzers.sharpe = btanalyzers.SharpeRatio(riskfreerate=0.03)
        self.analyzers.drawdown = btanalyzers.DrawDown()
        self.analyzers.volatility = btanalyzers.AnnualizedVolatility()
        self.analyzers.tradeanalyzer = btanalyzers.TradeAnalyzer()
    
    def next(self):
        # 记录每日组合价值
        self.dates.append(self.data.datetime.date())
        total_value = self.broker.getvalue()
        self.portfolio_values.append(total_value)
        
        # 检查是否是季度再平衡日
        current_date = self.data.datetime.date()
        if current_date in [q.date() for q in self.quarters]:
            self.rebalance_portfolio(current_date)
    
    def rebalance_portfolio(self, current_date):
        # 计算过去一个季度的收益率
        prev_quarter_start = current_date - pd.offsets.QuarterEnd()
        returns = {}
        
        for name, data in self.datas_dict.items():
            if name == '^GSPC':  # 跳过基准
                continue
                
            try:
                # 获取当前和上一季度初的价格
                idx = self.dates.index(current_date)
                prev_idx = self.dates.index(prev_quarter_start) if prev_quarter_start in self.dates else 0
                
                current_price = data.close[0]
                prev_price = data.close[-idx+prev_idx] if idx > prev_idx else data.close[0]
                
                if current_price > 0 and prev_price > 0:
                    returns[name] = (current_price - prev_price) / prev_price
            except (ValueError, IndexError):
                continue
        
        # 选择收益率最低的ETF
        if len(returns) >= self.p.num_assets:
            selected = sorted(returns.items(), key=lambda x: x[1])[:self.p.num_assets]
            selected_tickers = [x[0] for x in selected]
            selected_names = selected_tickers  # 简化处理，实际可以使用映射
            
            # 记录季度选择
            self.quarterly_selections.append((current_date, selected_tickers, selected_names))
        else:
            selected_tickers = []
            self.quarterly_selections.append((current_date, [], ["现金"]))
        
        # 卖出不在新选择中的持仓
        for name in list(self.current_holdings):
            if name not in selected_tickers:
                self.close(data=self.datas_dict[name])
                self.current_holdings.remove(name)
        
        # 计算当前总价值
        total_value = self.broker.getvalue()
        
        # 买入新增的ETF
        if selected_tickers:
            allocation = total_value / len(selected_tickers)
            
            # 调整现有持仓
            for name in set(self.current_holdings) & set(selected_tickers):
                data = self.datas_dict[name]
                current_value = self.getposition(data).size * data.close[0]
                delta = allocation - current_value
                
                if delta > 0:  # 需要买入更多
                    self.order_target_value(data=data, target=allocation)
                elif delta < 0:  # 需要卖出部分
                    self.order_target_value(data=data, target=allocation)
            
            # 买入新增ETF
            for name in set(selected_tickers) - set(self.current_holdings):
                data = self.datas_dict[name]
                self.order_target_value(data=data, target=allocation)
                self.current_holdings.add(name)
        
        # 记录季度表现
        if len(self.quarterly_performance) > 0:
            last_quarter_value = self.quarterly_performance[-1][1]
            quarter_return = (total_value - last_quarter_value) / last_quarter_value
        else:
            quarter_return = 0
        
        self.quarterly_performance.append((current_date, total_value, quarter_return))
    
    def stop(self):
        # 回测结束时打印结果
        print("\n季度选择记录及收益率:")
        for i, quarter in enumerate(self.quarterly_selections):
            date = quarter[0].strftime('%Y-%m-%d')
            tickers = ", ".join(quarter[1]) if quarter[1] else "现金"
            names = ", ".join(quarter[2]) if quarter[2] else "现金"
            q_return = self.quarterly_performance[i][2] * 100
            print(f"{date}: 选择基金 {tickers} ({names}) | 季度收益率: {q_return:.2f}%")
        
        # 打印分析结果
        self.print_analyzers()
    
    def print_analyzers(self):
        print("\n策略绩效指标:")
        print(f"初始资金: {self.p.initial_cash:,.2f}元")
        print(f"最终资金: {self.broker.getvalue():,.2f}元")
        
        # 总收益率
        rets = self.analyzers.returns.get_analysis()
        print(f"总收益率: {rets['rtot']*100:.2f}%")
        print(f"年化收益率: {rets['rnorm']*100:.2f}%")
        
        # 夏普比率
        sharpe = self.analyzers.sharpe.get_analysis()
        print(f"夏普比率: {sharpe['sharperatio']:.2f}")
        
        # 最大回撤
        drawdown = self.analyzers.drawdown.get_analysis()
        print(f"最大回撤: {drawdown['max']['drawdown']*100:.2f}%")
        
        # 年化波动率
        vol = self.analyzers.volatility.get_analysis()
        print(f"年化波动率: {vol['volatility']*100:.2f}%")
    pass

# 回测执行函数
def run_backtest():
    print("开始准备数据...")
    data_prep = DataPreparation()
    cerebro = data_prep.prepare_data()
    
    print("\n添加策略...")
    cerebro.addstrategy(QuarterlyReverseRotation)
    
    # 设置初始资金和佣金
    cerebro.broker.setcash(100000)
    cerebro.broker.setcommission(commission=0.001)
    
    # 添加分析器
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.03)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    
    print("\n开始回测...")
    print(f'初始资金: {cerebro.broker.getvalue():,.2f}元')
    
    results = cerebro.run()
    
    print(f'\n最终资金: {cerebro.broker.getvalue():,.2f}元')
    
    # 打印分析结果
    strat = results[0]
    print('\n=== 回测结果 ===')
    print(f"年化收益率: {strat.analyzers.returns.get_analysis()['rnorm']*100:.2f}%")
    print(f"夏普比率: {strat.analyzers.sharpe.get_analysis()['sharperatio']:.2f}")
    print(f"最大回撤: {strat.analyzers.drawdown.get_analysis()['max']['drawdown']*100:.2f}%")
    
    # 绘制结果
    cerebro.plot(style='candlestick')

if __name__ == '__main__':
    run_backtest()

开始准备数据...
正在获取 纳斯达克ETF(QQQ) 数据...


[*********************100%***********************]  1 of 1 completed


正在获取 标普500ETF(SPY) 数据...


[*********************100%***********************]  1 of 1 completed


正在获取 标普500指数(^GSPC) 数据...


[*********************100%***********************]  1 of 1 completed


正在获取 有色金属ETF(159980) 数据...
正在获取 恒生科技ETF(513010) 数据...
正在获取 恒生医药ETF(159892) 数据...

添加策略...

开始回测...
初始资金: 100,000.00元


AttributeError: 'tuple' object has no attribute 'lower'

In [5]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
from datetime import datetime
import os

# 定义定投策略
class DollarCostAveraging(bt.Strategy):
    params = (
        ('monthday', 1),  # 每月第1个交易日执行
        ('monthly_cash', 1000),  # 每月投入金额
    )

    def __init__(self):
        # 记录订单和执行价格
        self.order = None
        self.executed_prices = []
        self.executed_dates = []
        self.executed_shares = []

    def next(self):
        # 如果已有订单未完成，则不再新建订单
        if self.order:
            return

        # 检查是否是月初
        if not self._is_month_start():
            return

        # 计算可以购买的数量
        cash = self.broker.get_cash()
        if cash < self.p.monthly_cash:
            print(f'Not enough cash to invest: {cash} < {self.p.monthly_cash}')
            return

        # 创建买入订单
        self.order = self.buy(size=self.p.monthly_cash / self.data.close[0])

    def _is_month_start(self):
        # 检查是否是月初第一个交易日
        current_date = self.data.datetime.date(0)
        if current_date.day != self.p.monthday:
            return False
        
        # 确保是交易日
        if len(self.data) > 1:
            prev_date = self.data.datetime.date(-1)
            if prev_date.month == current_date.month:
                return False
        
        return True

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.executed_prices.append(order.executed.price)
                self.executed_dates.append(self.data.datetime.date(0))
                self.executed_shares.append(order.executed.size)
                print(
                    f'BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}, '
                    f'Size: {order.executed.size:.2f}'
                )
            self.order = None

# 下载并保存数据函数
def download_and_save_data():
    # 创建数据目录（如果不存在）
    data_dir = 'data'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # 下载标普500数据(SPY ETF)
    print("Downloading SPY data from Yahoo Finance...")
    data = yf.download('SPY', start='2021-01-01', end='2025-07-31')
    
    # 重命名列以符合backtrader要求
    data = data.rename(columns={
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Adj Close': 'adj_close',
        'Volume': 'volume'
    })
    
    # 确保列名是小写
    data.columns = [str(x).lower() for x in data.columns]
    
    # 保存为CSV文件
    csv_path = os.path.join(data_dir, 'SPY_2021_2025.csv')
    data.to_csv(csv_path)
    print(f"Data saved to: {csv_path}")
    
    return data

# 设置回测
def run_backtest(data):
    # 初始化cerebro引擎
    cerebro = bt.Cerebro()
    
    # 设置初始资金
    initial_cash = 10000
    cerebro.broker.set_cash(initial_cash)
    
    # 设置佣金 - 0.1% ... 除以100去掉百分号
    cerebro.broker.setcommission(commission=0.001)
    
    # 打印数据列信息用于调试
    print("\nData columns:", data.columns)
    print("Data shape:", data.shape)
    
    # 创建Data Feed
    data_feed = bt.feeds.PandasData(
        dataname=data,
        datetime=None,  # 默认使用索引作为datetime
        open=0,        # 列索引
        high=1,
        low=2,
        close=3,
        volume=4,
        openinterest=-1  # -1表示没有此列
    )
    
    # 添加数据到引擎
    cerebro.adddata(data_feed)
    
    # 添加策略
    cerebro.addstrategy(DollarCostAveraging)
    
    # 添加分析器
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    
    # 运行回测
    print('\nStarting Backtest...')
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    # 提取结果
    strat = results[0]
    
    # 计算年化收益率
    total_return = (cerebro.broker.getvalue() / initial_cash) - 1
    years = (datetime(2025, 7, 31) - datetime(2021, 1, 1)).days / 365.25
    annualized_return = (1 + total_return) ** (1 / years) - 1
    
    # 获取其他指标
    sharpe_ratio = strat.analyzers.sharpe.get_analysis()['sharperatio']
    max_drawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown'] / 100
    
    # 计算Alpha (假设基准回报是SPY本身，所以Alpha应为0)
    alpha = 0  # 简化处理
    
    # 获取年度回报 - 修正后的处理方式
    annual_returns = {}
    time_returns = strat.analyzers.time_return.get_analysis()
    for dt, return_ in time_returns.items():
        year = dt.year
        annual_returns[year] = return_
    
    # 打印结果
    print("\n=== Backtest Results ===")
    print(f"Initial Portfolio Value: ${initial_cash:,.2f}")
    print(f"Final Portfolio Value: ${cerebro.broker.getvalue():,.2f}")
    print(f"Total Return: {total_return * 100:.2f}%")
    print(f"Annualized Return: {annualized_return * 100:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    print("\nAnnual Returns:")
    for year, return_ in sorted(annual_returns.items()):
        print(f"{year}: {return_ * 100:.2f}%")
    
    # 可视化
    print("\nGenerating performance chart...")
    cerebro.plot(style='candlestick', iplot=False)

if __name__ == '__main__':
    # 下载并保存数据
    spy_data = download_and_save_data()
    
    # 运行回测
    run_backtest(spy_data)

[*********************100%***********************]  1 of 1 completed


Data saved to: data/SPY_2021_2025.csv

Data columns: Index(['('close', 'spy')', '('high', 'spy')', '('low', 'spy')',
       '('open', 'spy')', '('volume', 'spy')'],
      dtype='object')
Data shape: (1146, 5)

Starting Backtest...
Starting Portfolio Value: 10000.00
BUY EXECUTED, Price: 358.21, Cost: 1020.95, Comm: 1.02, Size: 2.85
BUY EXECUTED, Price: 362.90, Cost: 1002.46, Comm: 1.00, Size: 2.76
BUY EXECUTED, Price: 382.75, Cost: 1019.98, Comm: 1.02, Size: 2.66
BUY EXECUTED, Price: 395.91, Cost: 994.70, Comm: 0.99, Size: 2.51
BUY EXECUTED, Price: 409.86, Cost: 1011.31, Comm: 1.01, Size: 2.47
BUY EXECUTED, Price: 428.26, Cost: 1001.39, Comm: 1.00, Size: 2.34
BUY EXECUTED, Price: 406.36, Cost: 994.57, Comm: 0.99, Size: 2.45
BUY EXECUTED, Price: 437.89, Cost: 1003.48, Comm: 1.00, Size: 2.29
BUY EXECUTED, Price: 433.62, Cost: 990.82, Comm: 0.99, Size: 2.28
Not enough cash to invest: 951.3029760980039 < 1000
Not enough cash to invest: 951.3029760980039 < 1000
Not enough cash to invest: 951

<IPython.core.display.Javascript object>

In [13]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

# 数据下载和预处理
def download_data(tickers, start_date, end_date):
    data_dict = {}
    for ticker in tickers:
        print(f"Downloading {ticker} data...")
        data = yf.download(ticker, start=start_date, end=end_date)
        data.columns = [str(x).lower() for x in data.columns]  # 列名转为小写
        data_dict[ticker] = data
    return data_dict

# 资产列表
assets = {
    'SPY': '标普500',
    'QQQ': '纳斯达克',
    'GLD': '黄金',
    'ZS=F': '大豆',
    'SHY': '短期美债',
    'CL=F': '原油'
}

# 下载数据
start_date = '2021-01-01'
end_date = '2025-07-01'
data_dict = download_data(assets.keys(), start_date, end_date)

# 保存为CSV并创建backtrader数据
cerebro = bt.Cerebro()
cerebro.broker.setcash(10000.0)  # 初始资金1万美元

# 添加数据到cerebro
for ticker, data in data_dict.items():
    # 确保volume列索引为4
    data = data[['open', 'high', 'low', 'close', 'volume']]
    data_feed = btfeeds.PandasData(
        dataname=data,
        datetime=None,
        open=0,
        high=1,
        low=2,
        close=3,
        volume=4,
        openinterest=None
    )
    cerebro.adddata(data_feed, name=ticker)

# 定义策略
class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),  # 选择4个资产
        ('rebalance_month', [1, 4, 7, 10]),  # 季度调整月份(1月,4月,7月,10月)
    )

    def __init__(self):
        self.assets = list(assets.keys())
        self.asset_returns = {asset: [] for asset in self.assets}
        self.rebalance_dates = []
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        # 记录年度起始价值
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 计算年度回报
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        # 检查是否是季度调整月份的第一天
        if dt.month in self.params.rebalance_month and dt.day == 1:
            self.rebalance_portfolio(dt)
    
    def rebalance_portfolio(self, dt):
        # 计算上个季度的收益率
        quarter_returns = {}
        for i, asset in enumerate(self.assets):
            data = self.datas[i]
            # 获取上个季度的起始和结束价格
            lookback_period = 90  # 大约一个季度的交易日
            if len(data) > lookback_period:
                start_price = data.close[-lookback_period]
                end_price = data.close[0]
                return_pct = (end_price - start_price) / start_price * 100
                quarter_returns[asset] = return_pct
        
        if not quarter_returns:
            return
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        
        # 记录季度回报
        self.quarterly_returns[dt] = quarter_returns
        
        # 获取当前持仓
        current_positions = {}
        for asset in self.assets:
            data = self.getdatabyname(asset)
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[asset] = pos
        
        # 计算目标仓位价值
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25  # 每个资产25%
        
        # 调整仓位
        for asset in self.assets:
            data = self.getdatabyname(asset)
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset in selected_assets:
                # 在选中的资产中
                if asset in current_positions:
                    # 已经有仓位，检查是否需要调整
                    current_value = current_pos * data.close[0]
                    if abs(current_value - target_per_asset) / target_per_asset > 0.05:  # 允许5%的偏差
                        diff = target_per_asset - current_value
                        if diff > 0:
                            self.buy(data, size=diff / data.close[0])
                            self.log_trade(dt, asset, 'BUY', diff / data.close[0], data.close[0])
                        else:
                            self.sell(data, size=abs(diff) / data.close[0])
                            self.log_trade(dt, asset, 'SELL', abs(diff) / data.close[0], data.close[0])
                else:
                    # 没有仓位，买入
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset, 'BUY', size, data.close[0])
            else:
                # 不在选中的资产中，如果有持仓则卖出
                if asset in current_positions:
                    self.close(data)
                    self.log_trade(dt, asset, 'CLOSE', current_pos, data.close[0])
        
        self.rebalance_dates.append(dt)
    
    def log_trade(self, date, asset, action, size, price):
        self.trade_log.append({
            'Date': date,
            'Asset': asset,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 输出交易日志
        self.trade_df = pd.DataFrame(self.trade_log)
        
        # 计算年度回报
        self.annual_return_df = pd.DataFrame(list(self.annual_returns.items()), columns=['Year', 'Return(%)'])
        
        # 输出季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            row.update(returns)
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

# 添加策略
cerebro.addstrategy(QuarterlyContrarianRotation)

# 添加分析器
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual_return')
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.TimeReturn, _name='timereturn')

# 运行回测
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# 获取策略实例
strategy = results[0]

# 打印交易明细
print("\n交易明细:")
print(strategy.trade_df)

# 打印年度回报
print("\n年度回报:")
print(strategy.annual_return_df)

# 打印季度资产回报
print("\n季度资产回报:")
print(strategy.quarterly_return_df)

# 计算并打印性能指标
def calculate_metrics(strategy, start_date, end_date):
    # 获取分析器结果
    sharpe_ratio = strategy.analyzers.sharpe.get_analysis()['sharperatio']
    drawdown = strategy.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown['max']['drawdown']
    
    # 计算年化收益率
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    years = (end_dt - start_dt).days / 365.25
    initial_value = 10000.0
    final_value = cerebro.broker.getvalue()
    cagr = (final_value / initial_value) ** (1/years) - 1
    cagr_pct = cagr * 100
    
    # 计算Alpha (假设基准是SPY)
    spy_data = data_dict['SPY']
    spy_start = spy_data.iloc[0]['close']
    spy_end = spy_data.iloc[-1]['close']
    spy_return = (spy_end - spy_start) / spy_start
    spy_cagr = (1 + spy_return) ** (1/years) - 1
    alpha = cagr - spy_cagr
    
    # 创建指标DataFrame
    metrics = pd.DataFrame({
        'Metric': ['年化收益', '夏普比率', '最大回撤', 'Alpha'],
        'Value': [f"{cagr_pct:.2f}%", f"{sharpe_ratio:.2f}", f"{max_drawdown:.2f}%", f"{alpha:.4f}"]
    })
    
    return metrics

# 打印性能指标
print("\n性能指标:")
metrics = calculate_metrics(strategy, start_date, end_date)
print(metrics)

# 绘制结果
cerebro.plot(iplot=False)
plt.show()

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


KeyError: "None of [Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')] are in the [columns]"

In [30]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
from datetime import datetime
import os

class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),
        ('rebalance_month', [1, 4, 7, 10]),
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {self.asset_names}")
        
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        if dt.month in self.params.rebalance_month and dt.day == 1:
            self.rebalance_portfolio(dt)
    
    def rebalance_portfolio(self, dt):
        quarter_returns = {}
        for data in self.assets:
            lookback_period = 63
            if len(data) > lookback_period:
                start_price = data.close[-lookback_period]
                end_price = data.close[0]
                return_pct = (end_price - start_price) / start_price * 100
                quarter_returns[data._name] = return_pct
        
        if not quarter_returns or len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 数据不足，跳过调仓")
            return
        
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        print(f"{dt} 选中资产: {selected_assets}")
        
        self.quarterly_returns[dt] = quarter_returns
        
        current_positions = {}
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25
        
        for data in self.assets:
            asset_name = data._name
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset_name in selected_assets:
                if asset_name in current_positions:
                    if abs(current_value - target_per_asset) / target_per_asset > 0.05:
                        diff = target_per_asset - current_value
                        if diff > 0:
                            self.buy(data, size=diff/data.close[0])
                            self.log_trade(dt, asset_name, 'BUY', diff/data.close[0], data.close[0])
                        else:
                            self.sell(data, size=abs(diff)/data.close[0])
                            self.log_trade(dt, asset_name, 'SELL', abs(diff)/data.close[0], data.close[0])
                else:
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
            else:
                if asset_name in current_positions:
                    self.close(data)
                    self.log_trade(dt, asset_name, 'CLOSE', current_pos, data.close[0])
    
    def log_trade(self, date, asset, action, size, price):
        self.trade_log.append({
            'Date': date,
            'Asset': asset,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        self.trade_df = pd.DataFrame(self.trade_log)
        self.annual_return_df = pd.DataFrame(list(self.annual_returns.items()), columns=['Year', 'Return(%)'])
        
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            row.update(returns)
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

def download_data(tickers, start_date, end_date):
    data_dict = {}
    for ticker in tickers:
        print(f"正在下载 {ticker} 数据...")
        try:
            # 使用Ticker方式下载数据
            data = yf.Ticker(ticker).history(start=start_date, end=end_date)
            
            if data.empty:
                print(f"{ticker} 下载数据为空")
                continue
                
            # 规范列名 - 直接重命名
            data = data.rename(columns={
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume'
            })
            
            # 确保所有必需列存在
            required_cols = ['open', 'high', 'low', 'close']
            missing_cols = [col for col in required_cols if col not in data.columns]
            if missing_cols:
                print(f"{ticker} 缺少必要列: {missing_cols}")
                continue
                
            # 添加volume列如果不存在
            if 'volume' not in data.columns:
                data['volume'] = 0
                
            # 选择并重排列
            data = data[['open', 'high', 'low', 'close', 'volume']]
            
            # 重置索引并填充缺失值
            data = data.reset_index()
            data.fillna(method='ffill', inplace=True)
            data.set_index('Date', inplace=True)
            
            data_dict[ticker] = data
            print(f"{ticker} 数据下载成功，时间范围: {data.index[0].date()} 至 {data.index[-1].date()}")
        except Exception as e:
            print(f"下载 {ticker} 失败: {str(e)}")
    
    return data_dict

def run_backtest():
    # 使用更可靠的资产代码
    assets = {
        'SPY': '标普500',
        'QQQ': '纳斯达克',
        'GLD': '黄金',
        'SOYB': '大豆ETF',  # 替代ZS=F
        'SHY': '短期美债',
        'USO': '原油ETF'   # 替代CL=F
    }
    
    start_date = '2017-01-01'
    end_date = '2025-07-01'
    
    print("\n=== 开始下载数据 ===")
    data_dict = download_data(assets.keys(), start_date, end_date)
    
    if not data_dict:
        print("\n警告: 没有成功下载任何数据")
        print("尝试使用替代数据源...")
        # 这里可以添加使用其他数据源的代码
        return
    
    print("\n=== 初始化回测引擎 ===")
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    
    print("\n=== 添加数据 ===")
    for ticker, data in data_dict.items():
        try:
            # 确保列名是字符串
            data.columns = ['open', 'high', 'low', 'close', 'volume']
            
            data_feed = bt.feeds.PandasData(
                dataname=data,
                datetime=None,
                open=0,
                high=1,
                low=2,
                close=3,
                volume=4,
                openinterest=-1
            )
            cerebro.adddata(data_feed, name=ticker)
            print(f"成功添加 {ticker} 数据")
        except Exception as e:
            print(f"添加 {ticker} 数据失败: {str(e)}")
    
    if not cerebro.datas:
        print("\n错误: 没有有效数据可供回测")
        return
    
    print("\n=== 添加策略和分析器 ===")
    cerebro.addstrategy(QuarterlyContrarianRotation)
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    
    print('\n=== 开始回测 ===')
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    
    try:
        results = cerebro.run()
    except Exception as e:
        print(f"\n回测错误: {str(e)}")
        return
    
    if not results:
        print("\n回测未产生结果")
        return
    
    strat = results[0]
    print(f'\n最终资金: {cerebro.broker.getvalue():.2f}')
    
    # 计算年化收益率
    total_days = (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days
    years = total_days / 365.25
    total_return = (cerebro.broker.getvalue() / 10000) - 1
    annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算Alpha
    spy_return = (data_dict['SPY']['close'][-1] - data_dict['SPY']['close'][0]) / data_dict['SPY']['close'][0]
    spy_annualized = (1 + spy_return) ** (1 / years) - 1 if years > 0 else 0
    alpha = annualized_return - spy_annualized
    
    # 获取分析器结果
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown.get('max', {}).get('drawdown', 0) / 100
    
    print("\n=== 回测结果 ===")
    print(f"年化收益率: {annualized_return * 100:.2f}%")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"最大回撤: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    print("\n=== 交易明细 ===")
    if hasattr(strat, 'trade_df'):
        print(strat.trade_df.to_string())
    else:
        print("无交易记录")
    
    print("\n=== 年度回报 ===")
    if hasattr(strat, 'annual_return_df'):
        print(strat.annual_return_df.to_string())
    else:
        print("无年度回报数据")
    
    print("\n=== 季度资产回报 ===")
    if hasattr(strat, 'quarterly_return_df'):
        print(strat.quarterly_return_df.to_string())
    else:
        print("无季度回报数据")
    
    print("\n生成回测图表...")
    try:
        cerebro.plot(style='line', iplot=False)
    except Exception as e:
        print(f"绘图失败: {str(e)}")

if __name__ == '__main__':
    run_backtest()


=== 开始下载数据 ===
正在下载 SPY 数据...


/tmp/ipykernel_520390/3379873561.py:149: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


SPY 数据下载成功，时间范围: 2017-01-03 至 2025-06-30
正在下载 QQQ 数据...


/tmp/ipykernel_520390/3379873561.py:149: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


QQQ 数据下载成功，时间范围: 2017-01-03 至 2025-06-30
正在下载 GLD 数据...


...

SHY 数据下载成功，时间范围: 2017-01-03 至 2025-06-30
正在下载 USO 数据...


/tmp/ipykernel_520390/3379873561.py:149: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


USO 数据下载成功，时间范围: 2017-01-03 至 2025-06-30

=== 初始化回测引擎 ===

=== 添加数据 ===
成功添加 SPY 数据
成功添加 QQQ 数据
成功添加 GLD 数据
成功添加 SOYB 数据
成功添加 SHY 数据
成功添加 USO 数据

=== 添加策略和分析器 ===

=== 开始回测 ===
初始资金: 10000.00
策略初始化完成，可交易资产: ['SPY', 'QQQ', 'GLD', 'SOYB', 'SHY', 'USO']
2018-10-01 选中资产: ['GLD', 'SOYB', 'SHY', 'USO']
2019-04-01 选中资产: ['SOYB', 'GLD', 'SHY', 'SPY']
2019-07-01 选中资产: ['USO', 'SOYB', 'SHY', 'SPY']
2019-10-01 选中资产: ['USO', 'QQQ', 'SPY', 'SHY']
2020-04-01 选中资产: ['USO', 'SPY', 'QQQ', 'SOYB']
2020-07-01 选中资产: ['USO', 'SHY', 'SOYB', 'GLD']
2020-10-01 选中资产: ['USO', 'SHY', 'GLD', 'SPY']
2021-04-01 选中资产: ['GLD', 'SHY', 'QQQ', 'SPY']
2021-07-01 选中资产: ['SHY', 'GLD', 'SPY', 'QQQ']
2021-10-01 选中资产: ['SOYB', 'GLD', 'SHY', 'SPY']
2022-04-01 选中资产: ['QQQ', 'SPY', 'SHY', 'GLD']
2022-07-01 选中资产: ['QQQ', 'SPY', 'GLD', 'SOYB']
2024-04-01 选中资产: ['SOYB', 'SHY', 'QQQ', 'GLD']
2024-07-01 选中资产: ['SOYB', 'SHY', 'USO', 'GLD']
2024-10-01 选中资产: ['USO', 'SOYB', 'QQQ', 'SHY']
2025-04-01 选中资产: ['QQQ', 'SPY', 'SHY', 'SOYB']

最

/tmp/ipykernel_520390/3379873561.py:242: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  spy_return = (data_dict['SPY']['close'][-1] - data_dict['SPY']['close'][0]) / data_dict['SPY']['close'][0]


<IPython.core.display.Javascript object>

In [32]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import os

class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),
        ('rebalance_months', [1, 4, 7, 10]),  # 调仓月份
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {self.asset_names}")
        
        # 记录每个资产的最小长度
        self.min_length = min(len(d) for d in self.assets)
        print(f"最短数据长度: {self.min_length}")
        
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None  # 记录上次调仓日期

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < 65:  # 需要至少65天的历史数据
                return
                
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                self.rebalance_portfolio(dt)
                self.last_rebalance = dt

    def rebalance_portfolio(self, dt):
        print(f"\n=== {dt} 季度调仓 ===")
        
        # 计算上个季度的收益率
        quarter_returns = {}
        for data in self.assets:
            # 使用90天作为季度长度（更准确）
            lookback_period = 90
            if len(data) > lookback_period:
                start_idx = max(0, len(data) - lookback_period)
                start_price = data.close[start_idx]
                end_price = data.close[0]
                
                # 确保价格有效
                if start_price > 0 and end_price > 0:
                    return_pct = (end_price - start_price) / start_price * 100
                    quarter_returns[data._name] = return_pct
                    print(f"{data._name}: {return_pct:.2f}%")
        
        if not quarter_returns or len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 数据不足，跳过调仓")
            return
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        print(f"选中资产: {selected_assets}")
        
        self.quarterly_returns[dt] = quarter_returns
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25
        
        print(f"总资产: {total_value:.2f}, 目标每资产: {target_per_asset:.2f}")
        
        # 调整仓位
        for data in self.assets:
            asset_name = data._name
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset_name in selected_assets:
                if asset_name in current_positions:
                    # 调整仓位
                    diff = target_per_asset - current_value
                    if abs(diff) > 1:  # 避免微小交易
                        if diff > 0:
                            size = diff / data.close[0]
                            self.buy(data, size=size)
                            self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
                            print(f"增持 {asset_name}: {size:.2f} 股")
                        else:
                            size = abs(diff) / data.close[0]
                            self.sell(data, size=size)
                            self.log_trade(dt, asset_name, 'SELL', size, data.close[0])
                            print(f"减持 {asset_name}: {size:.2f} 股")
                else:
                    # 新买入
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
                    print(f"买入 {asset_name}: {size:.2f} 股")
            else:
                if asset_name in current_positions:
                    # 清仓
                    self.close(data)
                    self.log_trade(dt, asset_name, 'CLOSE', current_pos, data.close[0])
                    print(f"清仓 {asset_name}: {current_pos:.2f} 股")
    
    def log_trade(self, date, asset, action, size, price):
        self.trade_log.append({
            'Date': date,
            'Asset': asset,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                self.close(data)
                self.log_trade(dt, data._name, 'CLOSE', pos, data.close[0])
                print(f"清仓 {data._name}: {pos:.2f} 股")
        
        # 创建数据框
        self.trade_df = pd.DataFrame(self.trade_log)
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            row.update(returns)
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

def download_data(tickers, start_date, end_date):
    # 扩展开始日期以获取足够的历史数据
    extended_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=180)).strftime('%Y-%m-%d')
    
    data_dict = {}
    for ticker in tickers:
        print(f"正在下载 {ticker} 数据 ({extended_start} 至 {end_date})...")
        try:
            data = yf.Ticker(ticker).history(start=extended_start, end=end_date)
            
            if data.empty:
                print(f"{ticker} 下载数据为空")
                continue
                
            # 规范列名
            data = data.rename(columns={
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume'
            })
            
            # 确保所有必需列存在
            required_cols = ['open', 'high', 'low', 'close']
            if not all(col in data.columns for col in required_cols):
                print(f"{ticker} 缺少必要列")
                continue
                
            # 添加volume列如果不存在
            if 'volume' not in data.columns:
                data['volume'] = 0
                
            # 选择并重排列
            data = data[['open', 'high', 'low', 'close', 'volume']]
            
            # 重置索引并填充缺失值
            data = data.reset_index()
            data.fillna(method='ffill', inplace=True)
            data = data.dropna()
            data['Date'] = pd.to_datetime(data['Date'])
            data.set_index('Date', inplace=True)
            
            # 截取到实际开始日期
            data = data.loc[start_date:]
            
            data_dict[ticker] = data
            print(f"{ticker} 数据下载成功，{len(data)} 条记录，时间范围: {data.index[0].date()} 至 {data.index[-1].date()}")
        except Exception as e:
            print(f"下载 {ticker} 失败: {str(e)}")
    
    return data_dict

def run_backtest():
    assets = {
        'SPY': '标普500',
        'QQQ': '纳斯达克',
        'GLD': '黄金',
        'SOYB': '大豆ETF',
        'SHY': '短期美债',
        'USO': '原油ETF'
    }
    
    start_date = '2017-01-01'
    end_date = '2025-07-01'
    
    print("\n=== 开始下载数据 ===")
    data_dict = download_data(assets.keys(), start_date, end_date)
    
    if not data_dict:
        print("\n错误: 没有成功下载数据")
        return
    
    print("\n=== 初始化回测引擎 ===")
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    
    print("\n=== 添加数据 ===")
    for ticker, data in data_dict.items():
        try:
            # 确保列名正确
            data.columns = ['open', 'high', 'low', 'close', 'volume']
            
            # 检查数据长度
            if len(data) < 100:
                print(f"{ticker} 数据长度不足 ({len(data)}), 跳过")
                continue
                
            data_feed = bt.feeds.PandasData(
                dataname=data,
                fromdate=datetime.strptime(start_date, '%Y-%m-%d'),
                todate=datetime.strptime(end_date, '%Y-%m-%d'),
                datetime=None,
                open=0,
                high=1,
                low=2,
                close=3,
                volume=4,
                openinterest=-1
            )
            cerebro.adddata(data_feed, name=ticker)
            print(f"成功添加 {ticker} 数据 ({len(data)} 条记录)")
        except Exception as e:
            print(f"添加 {ticker} 数据失败: {str(e)}")
    
    if not cerebro.datas:
        print("\n错误: 没有有效数据")
        return
    
    print("\n=== 添加策略和分析器 ===")
    cerebro.addstrategy(QuarterlyContrarianRotation)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    
    print('\n=== 开始回测 ===')
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    
    try:
        results = cerebro.run()
    except Exception as e:
        print(f"\n回测错误: {str(e)}")
        return
    
    if not results:
        print("\n回测未产生结果")
        return
    
    strat = results[0]
    print(f'\n最终资金: {cerebro.broker.getvalue():.2f}')
    
    # 计算年化收益率
    total_days = (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days
    years = total_days / 365.25
    total_return = (cerebro.broker.getvalue() / 10000) - 1
    annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算Alpha
    spy_start = data_dict['SPY'].iloc[0]['close']
    spy_end = data_dict['SPY'].iloc[-1]['close']
    spy_return = (spy_end - spy_start) / spy_start if spy_start > 0 else 0
    spy_annualized = (1 + spy_return) ** (1 / years) - 1 if years > 0 else 0
    alpha = annualized_return - spy_annualized
    
    # 获取分析器结果
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown.get('max', {}).get('drawdown', 0) / 100
    
    print("\n=== 回测结果 ===")
    print(f"总天数: {total_days}")
    print(f"年化收益率: {annualized_return * 100:.2f}%")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"最大回撤: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    print("\n=== 交易明细 ===")
    if hasattr(strat, 'trade_df'):
        print(strat.trade_df.to_string())
        # 保存到CSV
        strat.trade_df.to_csv('trade_details.csv', index=False)
        print("交易明细已保存到 trade_details.csv")
    else:
        print("无交易记录")
    
    print("\n=== 年度回报 ===")
    if hasattr(strat, 'annual_return_df'):
        print(strat.annual_return_df.to_string())
        strat.annual_return_df.to_csv('annual_returns.csv', index=False)
        print("年度回报已保存到 annual_returns.csv")
    else:
        print("无年度回报数据")
    
    print("\n=== 季度资产回报 ===")
    if hasattr(strat, 'quarterly_return_df'):
        print(strat.quarterly_return_df.to_string())
        strat.quarterly_return_df.to_csv('quarterly_returns.csv', index=False)
        print("季度回报已保存到 quarterly_returns.csv")
    else:
        print("无季度回报数据")
    
    print("\n生成回测图表...")
    try:
        cerebro.plot(style='line', iplot=False)
    except Exception as e:
        print(f"绘图失败: {str(e)}")

if __name__ == '__main__':
    run_backtest()


=== 开始下载数据 ===
正在下载 SPY 数据 (2016-07-05 至 2025-07-01)...
SPY 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 QQQ 数据 (2016-07-05 至 2025-07-01)...
QQQ 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 GLD 数据 (2016-07-05 至 2025-07-01)...
GLD 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 SOYB 数据 (2016-07-05 至 2025-07-01)...
SOYB 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 SHY 数据 (2016-07-05 至 2025-07-01)...
SHY 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 USO 数据 (2016-07-05 至 2025-07-01)...
USO 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30

=== 初始化回测引擎 ===

=== 添加数据 ===
成功添加 SPY 数据 (2134 条记录)
成功添加 QQQ 数据 (2134 条记录)
成功添加 GLD 数据 (2134 条记录)
成功添加 SOYB 数据 (2134 条记录)
成功添加 SHY 数据 (2134 条记录)
成功添加 USO 数据 (2134 条记录)

=== 添加策略和分析器 ===

=== 开始回测 ===
初始资金: 10000.00


/tmp/ipykernel_520390/711484093.py:215: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/711484093.py:215: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/711484093.py:215: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/711484093.py:215: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/711484093.py:215: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a 

策略初始化完成，可交易资产: ['SPY', 'QQQ', 'GLD', 'SOYB', 'SHY', 'USO']
最短数据长度: 0

=== 2017-04-05 季度调仓 ===
2017-04-05 数据不足，跳过调仓

=== 2017-07-03 季度调仓 ===
SPY: -0.96%
QQQ: -4.56%
GLD: -5.36%
SOYB: 4.10%
SHY: -0.48%
USO: -2.63%
选中资产: ['GLD', 'QQQ', 'USO', 'SPY']
总资产: 10000.00, 目标每资产: 2500.00
买入 SPY: 11.75 股
买入 QQQ: 19.38 股
买入 GLD: 21.54 股
买入 USO: 32.48 股

=== 2017-10-02 季度调仓 ===
SPY: -8.62%
QQQ: -13.61%
GLD: -4.26%
SOYB: -4.58%
SHY: 0.76%
USO: -20.05%
选中资产: ['USO', 'QQQ', 'SPY', 'SOYB']
总资产: 10380.42, 目标每资产: 2595.10
减持 SPY: 0.10 股
减持 QQQ: 0.60 股
清仓 GLD: 21.54 股
买入 SOYB: 143.06 股
买入 USO: 31.77 股

=== 2018-01-02 季度调仓 ===
SPY: -6.75%
QQQ: -12.67%
GLD: 11.54%
SOYB: 13.11%
SHY: -0.32%
USO: -15.42%
选中资产: ['USO', 'QQQ', 'SPY', 'SHY']
总资产: 10770.62, 目标每资产: 2692.66
减持 SPY: 0.36 股
减持 QQQ: 0.92 股
清仓 SOYB: 143.06 股
买入 SHY: 37.40 股
买入 USO: 27.89 股

=== 2018-04-02 季度调仓 ===
SPY: -8.88%
QQQ: -10.33%
GLD: 0.62%
SOYB: 15.84%
SHY: -1.86%
USO: 6.79%
选中资产: ['QQQ', 'SPY', 'SHY', 'GLD']
总资产: 10621.54, 目标每资产: 2655.38
增持 SPY:

In [33]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import os
import numpy as np

class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),
        ('rebalance_months', [1, 4, 7, 10]),
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {self.asset_names}")
        
        # 记录每个资产的最小长度
        self.min_length = min(len(d) for d in self.assets)
        print(f"最短数据长度: {self.min_length}")
        
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None  # 记录上次调仓日期

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < 65:  # 需要至少65天的历史数据
                print(f"{dt} 数据不足，跳过调仓")
                return
                
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                self.rebalance_portfolio(dt)
                self.last_rebalance = dt

    def rebalance_portfolio(self, dt):
        print(f"\n=== {dt} 季度调仓 ===")
        
        # 计算上个季度的收益率
        quarter_returns = {}
        for data in self.assets:
            # 使用更稳健的方式计算季度回报
            lookback_period = min(90, len(data) - 1)  # 确保不超过数据长度
            if lookback_period > 60:  # 至少需要60天数据
                try:
                    # 使用更安全的数据访问方式
                    start_price = data.close[-lookback_period]
                    end_price = data.close[0]
                    
                    # 确保价格有效
                    if start_price > 0 and end_price > 0:
                        return_pct = (end_price - start_price) / start_price * 100
                        quarter_returns[data._name] = return_pct
                        print(f"{data._name}: {return_pct:.2f}%")
                except IndexError:
                    print(f"{data._name} 数据访问错误，跳过")
            else:
                print(f"{data._name} 数据不足 ({len(data)} bars)，跳过")
        
        if not quarter_returns or len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 有效资产不足 ({len(quarter_returns)} < {self.params.selection_count})，跳过调仓")
            return
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        print(f"选中资产: {selected_assets}")
        
        self.quarterly_returns[dt] = quarter_returns
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25
        
        print(f"总资产: {total_value:.2f}, 目标每资产: {target_per_asset:.2f}")
        
        # 调整仓位
        for data in self.assets:
            asset_name = data._name
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset_name in selected_assets:
                if asset_name in current_positions:
                    # 调整仓位
                    diff = target_per_asset - current_value
                    if abs(diff) > 1:  # 避免微小交易
                        if diff > 0:
                            size = diff / data.close[0]
                            self.buy(data, size=size)
                            self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
                            print(f"增持 {asset_name}: {size:.2f} 股")
                        else:
                            size = abs(diff) / data.close[0]
                            self.sell(data, size=size)
                            self.log_trade(dt, asset_name, 'SELL', size, data.close[0])
                            print(f"减持 {asset_name}: {size:.2f} 股")
                else:
                    # 新买入
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
                    print(f"买入 {asset_name}: {size:.2f} 股")
            else:
                if asset_name in current_positions:
                    # 清仓
                    self.close(data)
                    self.log_trade(dt, asset_name, 'CLOSE', current_pos, data.close[0])
                    print(f"清仓 {asset_name}: {current_pos:.2f} 股")
    
    def log_trade(self, date, asset, action, size, price):
        self.trade_log.append({
            'Date': date,
            'Asset': asset,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                try:
                    self.close(data)
                    self.log_trade(dt, data._name, 'CLOSE', pos, data.close[0])
                    print(f"清仓 {data._name}: {pos:.2f} 股")
                except Exception as e:
                    print(f"清仓 {data._name} 失败: {str(e)}")
        
        # 创建数据框
        self.trade_df = pd.DataFrame(self.trade_log)
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            row.update(returns)
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

def download_data(tickers, start_date, end_date):
    # 扩展开始日期以获取足够的历史数据
    extended_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=365)).strftime('%Y-%m-%d')
    
    data_dict = {}
    for ticker in tickers:
        print(f"正在下载 {ticker} 数据 ({extended_start} 至 {end_date})...")
        try:
            data = yf.download(ticker, start=extended_start, end=end_date, progress=False)
            
            if data.empty:
                print(f"{ticker} 下载数据为空")
                continue
                
            # 规范列名
            data = data.rename(columns={
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Volume': 'volume'
            })
            
            # 确保所有必需列存在
            required_cols = ['open', 'high', 'low', 'close']
            if not all(col in data.columns for col in required_cols):
                print(f"{ticker} 缺少必要列")
                continue
                
            # 添加volume列如果不存在
            if 'volume' not in data.columns:
                data['volume'] = 0
                
            # 选择并重排列
            data = data[['open', 'high', 'low', 'close', 'volume']]
            
            # 重置索引并填充缺失值
            data = data.reset_index()
            data.fillna(method='ffill', inplace=True)
            data = data.dropna()
            data['Date'] = pd.to_datetime(data['Date'])
            data.set_index('Date', inplace=True)
            
            # 截取到实际开始日期
            data = data.loc[start_date:]
            
            data_dict[ticker] = data
            print(f"{ticker} 数据下载成功，{len(data)} 条记录，时间范围: {data.index[0].date()} 至 {data.index[-1].date()}")
        except Exception as e:
            print(f"下载 {ticker} 失败: {str(e)}")
    
    return data_dict

def run_backtest():
    assets = {
        'SPY': '标普500',
        'QQQ': '纳斯达克',
        'GLD': '黄金',
        'SOYB': '大豆ETF',
        'SHY': '短期美债',
        'USO': '原油ETF'
    }
    
    start_date = '2017-01-01'
    end_date = '2025-07-01'
    
    print("\n=== 开始下载数据 ===")
    data_dict = download_data(assets.keys(), start_date, end_date)
    
    if not data_dict:
        print("\n错误: 没有成功下载数据")
        return
    
    print("\n=== 初始化回测引擎 ===")
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    
    print("\n=== 添加数据 ===")
    for ticker, data in data_dict.items():
        try:
            # 确保列名正确
            data.columns = ['open', 'high', 'low', 'close', 'volume']
            
            # 检查数据长度
            if len(data) < 100:
                print(f"{ticker} 数据长度不足 ({len(data)}), 跳过")
                continue
                
            data_feed = bt.feeds.PandasData(
                dataname=data,
                fromdate=datetime.strptime(start_date, '%Y-%m-%d'),
                todate=datetime.strptime(end_date, '%Y-%m-%d'),
                datetime=None,
                open=0,
                high=1,
                low=2,
                close=3,
                volume=4,
                openinterest=-1
            )
            cerebro.adddata(data_feed, name=ticker)
            print(f"成功添加 {ticker} 数据 ({len(data)} 条记录)")
        except Exception as e:
            print(f"添加 {ticker} 数据失败: {str(e)}")
    
    if not cerebro.datas:
        print("\n错误: 没有有效数据")
        return
    
    print("\n=== 添加策略和分析器 ===")
    cerebro.addstrategy(QuarterlyContrarianRotation)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    
    print('\n=== 开始回测 ===')
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    
    try:
        results = cerebro.run()
    except Exception as e:
        print(f"\n回测错误: {str(e)}")
        return
    
    if not results:
        print("\n回测未产生结果")
        return
    
    strat = results[0]
    print(f'\n最终资金: {cerebro.broker.getvalue():.2f}')
    
    # 计算年化收益率
    total_days = (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days
    years = total_days / 365.25
    total_return = (cerebro.broker.getvalue() / 10000) - 1
    annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算Alpha
    spy_start = data_dict['SPY'].iloc[0]['close']
    spy_end = data_dict['SPY'].iloc[-1]['close']
    spy_return = (spy_end - spy_start) / spy_start if spy_start > 0 else 0
    spy_annualized = (1 + spy_return) ** (1 / years) - 1 if years > 0 else 0
    alpha = annualized_return - spy_annualized
    
    # 获取分析器结果
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown.get('max', {}).get('drawdown', 0) / 100
    
    print("\n=== 回测结果 ===")
    print(f"总天数: {total_days}")
    print(f"年化收益率: {annualized_return * 100:.2f}%")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"最大回撤: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    print("\n=== 交易明细 ===")
    if hasattr(strat, 'trade_df'):
        print(strat.trade_df.to_string())
        # 保存到CSV
        strat.trade_df.to_csv('trade_details.csv', index=False)
        print("交易明细已保存到 trade_details.csv")
    else:
        print("无交易记录")
    
    print("\n=== 年度回报 ===")
    if hasattr(strat, 'annual_return_df'):
        print(strat.annual_return_df.to_string())
        strat.annual_return_df.to_csv('annual_returns.csv', index=False)
        print("年度回报已保存到 annual_returns.csv")
    else:
        print("无年度回报数据")
    
    print("\n=== 季度资产回报 ===")
    if hasattr(strat, 'quarterly_return_df'):
        print(strat.quarterly_return_df.to_string())
        strat.quarterly_return_df.to_csv('quarterly_returns.csv', index=False)
        print("季度回报已保存到 quarterly_returns.csv")
    else:
        print("无季度回报数据")
    
    print("\n生成回测图表...")
    try:
        cerebro.plot(style='line', iplot=False)
    except Exception as e:
        print(f"绘图失败: {str(e)}")

if __name__ == '__main__':
    run_backtest()


=== 开始下载数据 ===
正在下载 SPY 数据 (2016-01-02 至 2025-07-01)...


/tmp/ipykernel_520390/1023491429.py:225: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


SPY 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 QQQ 数据 (2016-01-02 至 2025-07-01)...


/tmp/ipykernel_520390/1023491429.py:225: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


QQQ 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 GLD 数据 (2016-01-02 至 2025-07-01)...


...

/tmp/ipykernel_520390/1023491429.py:225: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


SHY 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30
正在下载 USO 数据 (2016-01-02 至 2025-07-01)...


/tmp/ipykernel_520390/1023491429.py:225: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


USO 数据下载成功，2134 条记录，时间范围: 2017-01-03 至 2025-06-30

=== 初始化回测引擎 ===

=== 添加数据 ===
成功添加 SPY 数据 (2134 条记录)
成功添加 QQQ 数据 (2134 条记录)
成功添加 GLD 数据 (2134 条记录)
成功添加 SOYB 数据 (2134 条记录)
成功添加 SHY 数据 (2134 条记录)
成功添加 USO 数据 (2134 条记录)

=== 添加策略和分析器 ===

=== 开始回测 ===
初始资金: 10000.00
策略初始化完成，可交易资产: ['SPY', 'QQQ', 'GLD', 'SOYB', 'SHY', 'USO']
最短数据长度: 0
2017-01-03 数据不足，跳过调仓
2017-01-04 数据不足，跳过调仓
2017-01-05 数据不足，跳过调仓
2017-01-06 数据不足，跳过调仓
2017-01-09 数据不足，跳过调仓
2017-01-10 数据不足，跳过调仓
2017-01-11 数据不足，跳过调仓
2017-01-12 数据不足，跳过调仓
2017-01-13 数据不足，跳过调仓
2017-01-17 数据不足，跳过调仓
2017-01-18 数据不足，跳过调仓
2017-01-19 数据不足，跳过调仓
2017-01-20 数据不足，跳过调仓
2017-01-23 数据不足，跳过调仓
2017-01-24 数据不足，跳过调仓
2017-01-25 数据不足，跳过调仓
2017-01-26 数据不足，跳过调仓
2017-01-27 数据不足，跳过调仓
2017-01-30 数据不足，跳过调仓
2017-01-31 数据不足，跳过调仓
2017-04-03 数据不足，跳过调仓
2017-04-04 数据不足，跳过调仓

=== 2017-04-05 季度调仓 ===
SPY: 4.69%
QQQ: 10.63%
GLD: 8.28%
SOYB: -4.23%
SHY: 0.35%
USO: -6.64%
选中资产: ['USO', 'SOYB', 'SHY', 'SPY']
总资产: 10000.00, 目标每资产: 2500.00
买入 SPY: 12.18 股
买入 SOYB: 137.89 股
买入 SHY:

<IPython.core.display.Javascript object>

In [37]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta
import os
import numpy as np

# 定义资产
intl_etfs = {
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'  
}

china_etfs = {
    '159980': '有色金属etf',
    '513010': '恒生科技',
    '159892': '恒生医药',
    '159934': '黄金etf',
    '159985': '豆粕etf',
    '510880': '红利ETF',
    '516780': '稀土',
}

class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),
        ('rebalance_months', [1, 4, 7, 10]),
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        # 新增：代码到名称的映射
        self.code_to_name = {
            '159892': '恒生医药ETF',
            '516780': '稀土ETF',
            '513010': '恒生科技ETF',
            '510880': '红利ETF',
            '159980': '有色金属ETF',
            '159934': '黄金ETF',
            '159985': '豆粕ETF',
            'QQQ': '纳斯达克ETF',
            'SPY': '标普500ETF'
        }
        print(f"策略初始化完成，可交易资产: {[self.code_to_name.get(code, code) for code in self.asset_names]}")

        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < 65:
                return
                
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                self.rebalance_portfolio(dt)
                self.last_rebalance = dt

    def rebalance_portfolio(self, dt):
        print(f"\n=== {dt} 季度调仓 ===")
        
        # 计算上个季度的收益率
        quarter_returns = {}
        for data in self.assets:
            # 使用更稳健的方式计算季度回报
            lookback_period = min(90, len(data) - 1)
            if lookback_period > 60:
                try:
                    start_price = data.close[-lookback_period]
                    end_price = data.close[0]
                    
                    if start_price > 0 and end_price > 0:
                        return_pct = (end_price - start_price) / start_price * 100
                        quarter_returns[data._name] = return_pct
                        print(f"{data._name}: {return_pct:.2f}%")
                except IndexError:
                    print(f"{data._name} 数据访问错误，跳过")
        
        if not quarter_returns or len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 有效资产不足，跳过调仓")
            return
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        print(f"选中资产: {selected_assets}")
        
        self.quarterly_returns[dt] = quarter_returns
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25
        
        print(f"总资产: {total_value:.2f}, 目标每资产: {target_per_asset:.2f}")
        
        # 调整仓位
        for data in self.assets:
            asset_name = data._name
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset_name in selected_assets:
                if asset_name in current_positions:
                    # 调整仓位
                    diff = target_per_asset - current_value
                    if abs(diff) > 1:
                        if diff > 0:
                            size = diff / data.close[0]
                            self.buy(data, size=size)
                            self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
                            print(f"增持 {asset_name}: {size:.2f} 股")
                        else:
                            size = abs(diff) / data.close[0]
                            self.sell(data, size=size)
                            self.log_trade(dt, asset_name, 'SELL', size, data.close[0])
                            print(f"减持 {asset_name}: {size:.2f} 股")
                else:
                    # 新买入
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, 'BUY', size, data.close[0])
                    print(f"买入 {asset_name}: {size:.2f} 股")
            else:
                if asset_name in current_positions:
                    # 清仓
                    self.close(data)
                    self.log_trade(dt, asset_name, 'CLOSE', current_pos, data.close[0])
                    print(f"清仓 {asset_name}: {current_pos:.2f} 股")


    
    def log_trade(self, date, asset, action, size, price):
        # 交易日志中的代码替换
        display_name = self.code_to_name.get(asset, asset)
        self.trade_log.append({
            'Date': date,
            'Asset': display_name,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                try:
                    self.close(data)
                    self.log_trade(dt, data._name, 'CLOSE', pos, data.close[0])
                    print(f"清仓 {data._name}: {pos:.2f} 股")
                except Exception as e:
                    print(f"清仓 {data._name} 失败: {str(e)}")
        
        # 创建数据框
        self.trade_df = pd.DataFrame(self.trade_log)
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            row.update(returns)
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

def download_intl_etf(ticker, start_date, end_date):
    """使用yfinance下载国际ETF数据"""
    print(f"正在下载国际ETF {ticker} 数据 ({start_date} 至 {end_date})...")
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            print(f"{ticker} 下载数据为空")
            return None
            
        # 规范列名
        data = data.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
        
        # 确保所有必需列存在
        required_cols = ['open', 'high', 'low', 'close']
        if not all(col in data.columns for col in required_cols):
            print(f"{ticker} 缺少必要列")
            return None
            
        # 添加volume列如果不存在
        if 'volume' not in data.columns:
            data['volume'] = 0
            
        # 选择并重排列
        data = data[['open', 'high', 'low', 'close', 'volume']]
        
        # 重置索引并填充缺失值
        data = data.reset_index()
        data.fillna(method='ffill', inplace=True)
        data = data.dropna()
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        
        print(f"{ticker} 数据下载成功，{len(data)} 条记录，时间范围: {data.index[0].date()} 至 {data.index[-1].date()}")
        return data
    except Exception as e:
        print(f"下载国际ETF {ticker} 失败: {str(e)}")
        return None

def download_china_etf(ticker, start_date, end_date):
    """使用akshare的fund_etf_hist_em接口下载中国ETF数据"""
    print(f"正在下载中国ETF {ticker} 数据 ({start_date} 至 {end_date})...")
    try:
        # 使用akshare的fund_etf_hist_em接口
        df = ak.fund_etf_hist_em(symbol=ticker, period="daily")
        
        if df.empty:
            print(f"{ticker} 下载数据为空")
            return None
            
        # 规范列名
        df = df.rename(columns={
            '日期': 'date',
            '开盘': 'open',
            '收盘': 'close',
            '最高': 'high',
            '最低': 'low',
            '成交量': 'volume'
        })
        
        # 转换日期格式
        df['date'] = pd.to_datetime(df['date'])
        
        # 设置日期索引
        df.set_index('date', inplace=True)
        df.index.name = 'Date'
        
        # 截取指定时间范围
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        df = df.loc[start_dt:end_dt]
        
        if df.empty:
            print(f"{ticker} 在指定时间范围内无数据")
            return None
            
        # 确保数据类型正确
        for col in ['open', 'high', 'low', 'close']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
        
        # 处理可能的缺失值
        df.dropna(subset=['open', 'high', 'low', 'close'], inplace=True)
        
        # 前向填充缺失值
        df.fillna(method='ffill', inplace=True)
        
        # 选择并重排列
        df = df[['open', 'high', 'low', 'close', 'volume']]
        
        print(f"{ticker} 数据下载成功，{len(df)} 条记录，时间范围: {df.index[0].date()} 至 {df.index[-1].date()}")
        return df
    except Exception as e:
        print(f"下载中国ETF {ticker} 失败: {str(e)}")
        return None

def run_backtest():
    # 设置时间范围
    start_date = '2021-07-01'
    end_date = '2025-07-22'
    
    # 下载国际ETF数据
    intl_data = {}
    for ticker in intl_etfs:
        data = download_intl_etf(ticker, start_date, end_date)
        if data is not None:
            intl_data[ticker] = data
    
     # 下载中国ETF数据
    china_data = {}
    for ticker in china_etfs:
        data = download_china_etf(ticker, start_date, end_date)
        if data is not None:
            china_data[ticker] = data
    
    # 合并所有数据
    data_dict = {**intl_data, **china_data}
    
    if not data_dict:
        print("\n错误: 没有成功下载数据")
        return
    
    print("\n=== 初始化回测引擎 ===")
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    
    print("\n=== 添加数据 ===")
    for ticker, data in data_dict.items():
        try:
            # 确保列名正确
            data.columns = ['open', 'high', 'low', 'close', 'volume']
            
            # 检查数据长度
            if len(data) < 100:
                print(f"{ticker} 数据长度不足 ({len(data)}), 跳过")
                continue
                
            data_feed = bt.feeds.PandasData(
                dataname=data,
                fromdate=datetime.strptime(start_date, '%Y-%m-%d'),
                todate=datetime.strptime(end_date, '%Y-%m-%d'),
                datetime=None,
                open=0,
                high=1,
                low=2,
                close=3,
                volume=4,
                openinterest=-1
            )
            cerebro.adddata(data_feed, name=ticker)
            print(f"成功添加 {ticker} 数据 ({len(data)} 条记录)")
        except Exception as e:
            print(f"添加 {ticker} 数据失败: {str(e)}")
    
    if not cerebro.datas:
        print("\n错误: 没有有效数据")
        return
    
    print("\n=== 添加策略和分析器 ===")
    cerebro.addstrategy(QuarterlyContrarianRotation)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    
    print('\n=== 开始回测 ===')
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    
    try:
        results = cerebro.run()
    except Exception as e:
        print(f"\n回测错误: {str(e)}")
        return
    
    if not results:
        print("\n回测未产生结果")
        return
    
    strat = results[0]
    print(f'\n最终资金: {cerebro.broker.getvalue():.2f}')
    
    # 计算年化收益率
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    total_days = (end_dt - start_dt).days
    years = total_days / 365.25
    total_return = (cerebro.broker.getvalue() / 10000) - 1
    annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算Alpha (使用SPY作为基准)
    if 'SPY' in data_dict:
        spy_start = data_dict['SPY'].iloc[0]['close']
        spy_end = data_dict['SPY'].iloc[-1]['close']
        spy_return = (spy_end - spy_start) / spy_start if spy_start > 0 else 0
        spy_annualized = (1 + spy_return) ** (1 / years) - 1 if years > 0 else 0
        alpha = annualized_return - spy_annualized
    else:
        alpha = 0
    
    # 获取分析器结果
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown.get('max', {}).get('drawdown', 0) / 100
    
    print("\n=== 回测结果 ===")
    print(f"总天数: {total_days}")
    print(f"年化收益率: {annualized_return * 100:.2f}%")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"最大回撤: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    print("\n=== 交易明细 ===")
    if hasattr(strat, 'trade_df'):
        print(strat.trade_df.to_string())
        strat.trade_df.to_csv('trade_details.csv', index=False)
        print("交易明细已保存到 trade_details.csv")
    else:
        print("无交易记录")
    
    print("\n=== 年度回报 ===")
    if hasattr(strat, 'annual_return_df'):
        print(strat.annual_return_df.to_string())
        strat.annual_return_df.to_csv('annual_returns.csv', index=False)
        print("年度回报已保存到 annual_returns.csv")
    else:
        print("无年度回报数据")
    
    print("\n=== 季度资产回报 ===")
    if hasattr(strat, 'quarterly_return_df'):
        print(strat.quarterly_return_df.to_string())
        strat.quarterly_return_df.to_csv('quarterly_returns.csv', index=False)
        print("季度回报已保存到 quarterly_returns.csv")
    else:
        print("无季度回报数据")
    
    print("\n生成回测图表...")
    try:
        cerebro.plot(style='line', iplot=False)
    except Exception as e:
        print(f"绘图失败: {str(e)}")

if __name__ == '__main__':
    run_backtest()

正在下载国际ETF QQQ 数据 (2021-07-01 至 2025-07-22)...
QQQ 数据下载成功，1017 条记录，时间范围: 2021-07-01 至 2025-07-21
正在下载国际ETF SPY 数据 (2021-07-01 至 2025-07-22)...
SPY 数据下载成功，1017 条记录，时间范围: 2021-07-01 至 2025-07-21
正在下载中国ETF 159980 数据 (2021-07-01 至 2025-07-22)...


/tmp/ipykernel_520390/607802255.py:245: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/607802255.py:245: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/607802255.py:302: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/607802255.py:302: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


159980 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 513010 数据 (2021-07-01 至 2025-07-22)...
513010 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 159892 数据 (2021-07-01 至 2025-07-22)...


/tmp/ipykernel_520390/607802255.py:302: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


159892 数据下载成功，913 条记录，时间范围: 2021-10-19 至 2025-07-22
正在下载中国ETF 159934 数据 (2021-07-01 至 2025-07-22)...


...

/tmp/ipykernel_520390/607802255.py:302: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


159985 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 510880 数据 (2021-07-01 至 2025-07-22)...


/tmp/ipykernel_520390/607802255.py:302: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/tmp/ipykernel_520390/607802255.py:302: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


510880 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 516780 数据 (2021-07-01 至 2025-07-22)...
516780 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22

=== 初始化回测引擎 ===

=== 添加数据 ===
成功添加 QQQ 数据 (1017 条记录)
成功添加 SPY 数据 (1017 条记录)
成功添加 159980 数据 (984 条记录)
成功添加 513010 数据 (984 条记录)
成功添加 159892 数据 (913 条记录)
成功添加 159934 数据 (984 条记录)
成功添加 159985 数据 (984 条记录)
成功添加 510880 数据 (984 条记录)
成功添加 516780 数据 (984 条记录)

=== 添加策略和分析器 ===

=== 开始回测 ===
初始资金: 10000.00
策略初始化完成，可交易资产: ['纳斯达克ETF', '标普500ETF', '有色金属ETF', '恒生科技ETF', '恒生医药ETF', '黄金ETF', '豆粕ETF', '红利ETF', '稀土ETF']

=== 2021-10-19 季度调仓 ===
QQQ: 6.01%
SPY: 5.03%
159980: 19.21%
513010: -20.30%
159934: -0.97%
159985: -9.72%
510880: 8.97%
516780: 42.02%
选中资产: ['513010', '159985', '159934', 'SPY']
总资产: 10000.00, 目标每资产: 2500.00
买入 SPY: 5.85 股
买入 513010: 3105.59 股
买入 159934: 701.85 股
买入 159985: 2120.44 股

=== 2022-01-03 季度调仓 ===
QQQ: 7.43%
SPY: 7.13%
159980: 7.03%
513010: -10.60%
159934: -0.41%
159985: -6.94%
510880: 0.00%
516780: 10.85%
选中资产: ['513010',

<IPython.core.display.Javascript object>

In [2]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
import akshare as ak
from datetime import datetime, timedelta
import os
import numpy as np
import warnings
import time

warnings.filterwarnings('ignore')

# 定义资产
intl_etfs = {
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'  
}

china_etfs = {
    '159980': '有色金属etf',
    '513010': '恒生科技',
    '159892': '恒生医药',
    '159934': '黄金etf',
    '159985': '豆粕etf',
    '510880': '红利ETF',
    '516780': '稀土',
}

# 资产名称映射
asset_name_map = {
    '159980': '有色金属ETF',
    '513010': '恒生科技ETF',
    '159892': '恒生医药ETF',
    '159934': '黄金ETF',
    '159985': '豆粕ETF',
    '510880': '红利ETF',
    '516780': '稀土ETF',
    'QQQ': '纳斯达克ETF',
    'SPY': '标普500ETF'
}

class QuarterlyContrarianRotation(bt.Strategy):
    params = (
        ('selection_count', 4),
        ('rebalance_months', [1, 4, 7, 10]),
        ('quarter_trading_days', 63),  # 大约一个季度的交易日数
    )

    def __init__(self):
        self.assets = self.datas
        self.asset_names = [d._name for d in self.datas]
        print(f"策略初始化完成，可交易资产: {[asset_name_map.get(name, name) for name in self.asset_names]}")
        
        self.trade_log = []
        self.quarterly_returns = {}
        self.current_year = None
        self.year_start_value = None
        self.annual_returns = {}
        self.last_rebalance = None

    def next(self):
        dt = self.datas[0].datetime.date(0)
        
        # 初始化年度回报
        if self.current_year is None:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 年度更替
        if dt.year != self.current_year:
            self.current_year = dt.year
            self.year_start_value = self.broker.getvalue()
            self.annual_returns[self.current_year] = 0
        
        # 更新年度回报
        self.annual_returns[self.current_year] = (self.broker.getvalue() / self.year_start_value - 1) * 100
        
        # 检查是否是调仓月份
        if dt.month in self.params.rebalance_months:
            # 确保有足够的历史数据
            if len(self.data0) < self.params.quarter_trading_days + 5:
                return
                
            # 检查是否是该月第一次出现
            if self.last_rebalance is None or self.last_rebalance.month != dt.month:
                self.rebalance_portfolio(dt)
                self.last_rebalance = dt

    def rebalance_portfolio(self, dt):
        print(f"\n=== {dt} 季度调仓 ===")
        
        # 计算上个季度的收益率（使用固定交易日间隔）
        quarter_returns = {}
        lookback = self.params.quarter_trading_days
        
        for data in self.assets:
            asset_name = data._name
            display_name = asset_name_map.get(asset_name, asset_name)
            
            # 确保有足够的历史数据
            if len(data) < lookback + 1:
                # print(f"{display_name} 数据不足，跳过")
                continue
                
            try:
                # 获取价格 - 使用固定交易日间隔
                start_price = data.close[-lookback]
                end_price = data.close[0]
                
                if start_price > 0 and end_price > 0:
                    return_pct = (end_price - start_price) / start_price * 100
                    quarter_returns[asset_name] = return_pct
                    # 获取实际日期用于调试
                    start_date = data.datetime.date(-lookback)
                    print(f"{display_name}: {return_pct:.2f}% (日期: {start_date} -> {dt})")
                else:
                    print(f"{display_name} 价格无效: start={start_price}, end={end_price}")
            except Exception as e:
                print(f"{display_name} 数据访问错误: {str(e)}")
        
        if len(quarter_returns) < self.params.selection_count:
            print(f"{dt} 有效资产不足({len(quarter_returns)}), 需要{self.params.selection_count}个, 跳过调仓")
            return
        
        # 按收益率升序排序，选择表现最差的4个
        sorted_assets = sorted(quarter_returns.items(), key=lambda x: x[1])
        selected_assets = [x[0] for x in sorted_assets[:self.params.selection_count]]
        selected_names = [asset_name_map.get(a, a) for a in selected_assets]
        print(f"选中资产: {selected_names}")
        
        self.quarterly_returns[dt] = quarter_returns
        
        # 获取当前持仓
        current_positions = {}
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                current_positions[data._name] = pos
        
        total_value = self.broker.getvalue()
        target_per_asset = total_value * 0.25
        
        print(f"总资产: {total_value:.2f}, 目标每资产: {target_per_asset:.2f}")
        
        # 调整仓位
        for data in self.assets:
            asset_name = data._name
            display_name = asset_name_map.get(asset_name, asset_name)
            current_pos = self.getposition(data).size
            current_value = current_pos * data.close[0] if current_pos > 0 else 0
            
            if asset_name in selected_assets:
                if asset_name in current_positions:
                    # 调整仓位
                    diff = target_per_asset - current_value
                    if abs(diff) > 1:
                        if diff > 0:
                            size = diff / data.close[0]
                            self.buy(data, size=size)
                            self.log_trade(dt, asset_name, display_name, 'BUY', size, data.close[0])
                            print(f"增持 {display_name}: {size:.2f} 股")
                        else:
                            size = abs(diff) / data.close[0]
                            self.sell(data, size=size)
                            self.log_trade(dt, asset_name, display_name, 'SELL', size, data.close[0])
                            print(f"减持 {display_name}: {size:.2f} 股")
                else:
                    # 新买入
                    size = target_per_asset / data.close[0]
                    self.buy(data, size=size)
                    self.log_trade(dt, asset_name, display_name, 'BUY', size, data.close[0])
                    print(f"买入 {display_name}: {size:.2f} 股")
            else:
                if asset_name in current_positions:
                    # 清仓
                    self.close(data)
                    self.log_trade(dt, asset_name, display_name, 'CLOSE', current_pos, data.close[0])
                    print(f"清仓 {display_name}: {current_pos:.2f} 股")
    
    def log_trade(self, date, asset, display_name, action, size, price):
        self.trade_log.append({
            'Date': date,
            'Asset': display_name,
            'Action': action,
            'Size': size,
            'Price': price,
            'Value': size * price
        })
    
    def stop(self):
        # 在结束时处理持仓
        dt = self.datas[0].datetime.date(0)
        print(f"\n结束日期 {dt} 处理持仓")
        
        # 清空所有持仓
        for data in self.assets:
            pos = self.getposition(data).size
            if pos > 0:
                try:
                    display_name = asset_name_map.get(data._name, data._name)
                    self.close(data)
                    self.log_trade(dt, data._name, display_name, 'CLOSE', pos, data.close[0])
                    print(f"清仓 {display_name}: {pos:.2f} 股")
                except Exception as e:
                    print(f"清仓 {data._name} 失败: {str(e)}")
        
        # 创建数据框
        self.trade_df = pd.DataFrame(self.trade_log)
        
        # 计算年度回报
        years = sorted(set(int(y) for y in self.annual_returns.keys()))
        annual_returns = [self.annual_returns.get(y, 0) for y in years]
        self.annual_return_df = pd.DataFrame({
            'Year': years,
            'Return(%)': annual_returns
        })
        
        # 季度回报
        quarter_returns = []
        for date, returns in self.quarterly_returns.items():
            row = {'Date': date}
            # 使用显示名称
            for asset, ret in returns.items():
                display_name = asset_name_map.get(asset, asset)
                row[display_name] = ret
            quarter_returns.append(row)
        self.quarterly_return_df = pd.DataFrame(quarter_returns)

def download_intl_etf(ticker, start_date, end_date):
    """使用yfinance下载国际ETF数据"""
    print(f"正在下载国际ETF {ticker} 数据 ({start_date} 至 {end_date})...")
    try:
        # 增加一周缓冲期
        adjusted_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')
        data = yf.download(ticker, start=adjusted_start, end=end_date, progress=False)
        
        if data.empty:
            print(f"{ticker} 下载数据为空")
            return None
            
        # 规范列名
        data = data.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
        
        # 确保所有必需列存在
        required_cols = ['open', 'high', 'low', 'close']
        if not all(col in data.columns for col in required_cols):
            print(f"{ticker} 缺少必要列")
            return None
            
        # 添加volume列如果不存在
        if 'volume' not in data.columns:
            data['volume'] = 0
            
        # 选择并重排列
        data = data[['open', 'high', 'low', 'close', 'volume']]
        
        # 重置索引并填充缺失值
        data = data.reset_index()
        data.fillna(method='ffill', inplace=True)
        data = data.dropna()
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        
        # 截取指定时间范围
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        data = data.loc[start_dt:end_dt]
        
        print(f"{ticker} 数据下载成功，{len(data)} 条记录，时间范围: {data.index[0].date()} 至 {data.index[-1].date()}")
        return data
    except Exception as e:
        print(f"下载国际ETF {ticker} 失败: {str(e)}")
        return None

def download_china_etf(ticker, start_date, end_date, retry=3):
    """使用akshare下载中国ETF数据，带重试机制"""
    print(f"正在下载中国ETF {ticker} 数据 ({start_date} 至 {end_date})...")
    for attempt in range(retry):
        try:
            # 增加一周缓冲期
            adjusted_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y%m%d')
            adjusted_end = (datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=7)).strftime('%Y%m%d')
            
            # 使用akshare的fund_etf_hist_em接口
            df = ak.fund_etf_hist_em(symbol=ticker, period="daily", 
                                    start_date=adjusted_start, 
                                    end_date=adjusted_end)
            
            if df.empty:
                print(f"{ticker} 下载数据为空")
                return None
                
            # 规范列名
            df = df.rename(columns={
                '日期': 'date',
                '开盘': 'open',
                '收盘': 'close',
                '最高': 'high',
                '最低': 'low',
                '成交量': 'volume'
            })
            
            # 转换日期格式
            df['date'] = pd.to_datetime(df['date'])
            
            # 设置日期索引
            df.set_index('date', inplace=True)
            df.index.name = 'Date'
            
            # 截取指定时间范围
            start_dt = pd.to_datetime(start_date)
            end_dt = pd.to_datetime(end_date)
            df = df.loc[start_dt:end_dt]
            
            if df.empty:
                print(f"{ticker} 在指定时间范围内无数据")
                return None
                
            # 确保数据类型正确
            for col in ['open', 'high', 'low', 'close']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df['volume'] = pd.to_numeric(df['volume'], errors='coerce')
            
            # 处理可能的缺失值
            df.dropna(subset=['open', 'high', 'low', 'close'], inplace=True)
            
            # 前向填充缺失值
            df.fillna(method='ffill', inplace=True)
            
            # 选择并重排列
            df = df[['open', 'high', 'low', 'close', 'volume']]
            
            print(f"{ticker} 数据下载成功，{len(df)} 条记录，时间范围: {df.index[0].date()} 至 {df.index[-1].date()}")
            return df
        except Exception as e:
            if attempt < retry - 1:
                wait_time = (attempt + 1) * 5
                print(f"下载 {ticker} 失败，{wait_time}秒后重试... ({str(e)})")
                time.sleep(wait_time)
            else:
                print(f"下载中国ETF {ticker} 失败: {str(e)}")
                return None

def run_backtest():
    # 设置时间范围
    start_date = '2021-07-01'
    end_date = '2025-07-22'
    
    # 下载国际ETF数据
    intl_data = {}
    for ticker in intl_etfs:
        data = download_intl_etf(ticker, start_date, end_date)
        if data is not None:
            intl_data[ticker] = data
    
    # 下载中国ETF数据
    china_data = {}
    for ticker in china_etfs:
        # 对于159892，调整开始日期
        if ticker == '159892':
            adjusted_start_date = '2021-10-01'  # 早于实际开始日期确保有足够数据
        else:
            adjusted_start_date = start_date
            
        data = download_china_etf(ticker, adjusted_start_date, end_date)
        if data is not None:
            china_data[ticker] = data
    
    # 合并所有数据
    data_dict = {**intl_data, **china_data}
    
    if not data_dict:
        print("\n错误: 没有成功下载数据")
        return
    
    print("\n=== 初始化回测引擎 ===")
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    
    print("\n=== 添加数据 ===")
    for ticker, data in data_dict.items():
        try:
            # 确保列名正确
            data.columns = ['open', 'high', 'low', 'close', 'volume']
            
            # 检查数据长度
            if len(data) < 100:
                print(f"{ticker} 数据长度不足 ({len(data)}), 跳过")
                continue
                
            data_feed = bt.feeds.PandasData(
                dataname=data,
                fromdate=datetime.strptime(start_date, '%Y-%m-%d'),
                todate=datetime.strptime(end_date, '%Y-%m-%d'),
                datetime=None,
                open=0,
                high=1,
                low=2,
                close=3,
                volume=4,
                openinterest=-1
            )
            cerebro.adddata(data_feed, name=ticker)
            print(f"成功添加 {ticker} 数据 ({len(data)} 条记录)")
        except Exception as e:
            print(f"添加 {ticker} 数据失败: {str(e)}")
    
    if not cerebro.datas:
        print("\n错误: 没有有效数据")
        return
    
    print("\n=== 添加策略和分析器 ===")
    cerebro.addstrategy(QuarterlyContrarianRotation)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True)
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Years)
    
    print('\n=== 开始回测 ===')
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    
    try:
        results = cerebro.run()
    except Exception as e:
        print(f"\n回测错误: {str(e)}")
        return
    
    if not results:
        print("\n回测未产生结果")
        return
    
    strat = results[0]
    print(f'\n最终资金: {cerebro.broker.getvalue():.2f}')
    
    # 计算年化收益率
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    total_days = (end_dt - start_dt).days
    years = total_days / 365.25
    total_return = (cerebro.broker.getvalue() / 10000) - 1
    annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算Alpha (使用SPY作为基准)
    if 'SPY' in data_dict:
        spy_start = data_dict['SPY'].iloc[0]['close']
        spy_end = data_dict['SPY'].iloc[-1]['close']
        spy_return = (spy_end - spy_start) / spy_start if spy_start > 0 else 0
        spy_annualized = (1 + spy_return) ** (1 / years) - 1 if years > 0 else 0
        alpha = annualized_return - spy_annualized
    else:
        alpha = 0
    
    # 获取分析器结果
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0)
    drawdown = strat.analyzers.drawdown.get_analysis()
    max_drawdown = drawdown.get('max', {}).get('drawdown', 0) / 100
    
    print("\n=== 回测结果 ===")
    print(f"总天数: {total_days}")
    print(f"年化收益率: {annualized_return * 100:.2f}%")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"最大回撤: {max_drawdown * 100:.2f}%")
    print(f"Alpha: {alpha * 100:.2f}%")
    
    print("\n=== 交易明细 ===")
    if hasattr(strat, 'trade_df'):
        print(strat.trade_df.to_string())
        strat.trade_df.to_csv('trade_details.csv', index=False)
        print("交易明细已保存到 trade_details.csv")
    else:
        print("无交易记录")
    
    print("\n=== 年度回报 ===")
    if hasattr(strat, 'annual_return_df'):
        print(strat.annual_return_df.to_string())
        strat.annual_return_df.to_csv('annual_returns.csv', index=False)
        print("年度回报已保存到 annual_returns.csv")
    else:
        print("无年度回报数据")
    
    print("\n=== 季度资产回报 ===")
    if hasattr(strat, 'quarterly_return_df'):
        print(strat.quarterly_return_df.to_string())
        strat.quarterly_return_df.to_csv('quarterly_returns.csv', index=False)
        print("季度回报已保存到 quarterly_returns.csv")
    else:
        print("无季度回报数据")
    
    print("\n生成回测图表...")
    try:
        cerebro.plot(style='line', iplot=False)
    except Exception as e:
        print(f"绘图失败: {str(e)}")

if __name__ == '__main__':
    run_backtest()

正在下载国际ETF QQQ 数据 (2021-07-01 至 2025-07-22)...
QQQ 数据下载成功，1017 条记录，时间范围: 2021-07-01 至 2025-07-21
正在下载国际ETF SPY 数据 (2021-07-01 至 2025-07-22)...
SPY 数据下载成功，1017 条记录，时间范围: 2021-07-01 至 2025-07-21
正在下载中国ETF 159980 数据 (2021-07-01 至 2025-07-22)...
159980 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 513010 数据 (2021-07-01 至 2025-07-22)...
513010 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 159892 数据 (2021-10-01 至 2025-07-22)...
159892 数据下载成功，913 条记录，时间范围: 2021-10-19 至 2025-07-22
正在下载中国ETF 159934 数据 (2021-07-01 至 2025-07-22)...
159934 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 159985 数据 (2021-07-01 至 2025-07-22)...
159985 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 510880 数据 (2021-07-01 至 2025-07-22)...
510880 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22
正在下载中国ETF 516780 数据 (2021-07-01 至 2025-07-22)...
516780 数据下载成功，984 条记录，时间范围: 2021-07-01 至 2025-07-22

=== 初始化回测引擎 ===

=== 添加数据 ===
成功添加 QQQ 数据 (1017 条记录)
成功添加 SPY 数据 (1017 条记录)
成功添加 159980 数据 (984 条记录)

In [3]:
# 导入所需库
import yfinance as yf
import akshare as ak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置回测日期范围
end_date = datetime(2025, 7, 22)
start_date = end_date - timedelta(days=4*365)

# ================== 数据获取部分 ==================

# 获取国际ETF数据 (yfinance) - 加入SPY(标普500ETF)
intl_etfs = {
    'QQQ': '纳斯达克ETF',
    # 'SOYB': '大豆ETF',
    # 'GLD': '黄金ETF',
    # "BTC-USD":"Bitcoin",
    # 'SHY': 'usbond-ETF',
    'SPY': '标普500ETF'  # 新增SPY
}

# 获取国际ETF数据 (yfinance) - 加入SPY(标普500ETF)
intl_data = {}
for ticker, name in intl_etfs.items():
    print(f"获取{ticker}数据...")
    # 增加一周缓冲期
    adjusted_start_date = start_date + timedelta(days=7)
    df = yf.download(ticker, start=adjusted_start_date, end=end_date)
    if not df.empty:
        intl_data[ticker] = df[['Close']].rename(columns={'Close': 'close'})

# 获取标普500数据作为基准
print("获取标普500指数数据...")
# 增加一周缓冲期
adjusted_start_date = start_date + timedelta(days=7)
sp500 = yf.download('^GSPC', start=adjusted_start_date, end=end_date)[['Close']].rename(columns={'Close': 'close'})

# 获取中国ETF数据 (AKShare)
def fetch_etf_data_ak(symbol, start_date, end_date):
    """通过AKShare获取ETF数据"""
    try:
        # 增加一周缓冲期
        adjusted_start_date = start_date + timedelta(days=7)
        df = ak.fund_etf_hist_em(symbol=symbol, period="daily", 
                                start_date=adjusted_start_date.strftime("%Y%m%d"), 
                                end_date=end_date.strftime("%Y%m%d"), 
                                adjust="hfq")
        df = df.rename(columns={'日期':'date', '收盘':'close'})
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        return df[['close']]
    except Exception as e:
        print(f"获取{symbol}数据失败: {e}")
        return None

# 中国ETF列表
china_etfs = {
    # '159806': '新能源车etf',
    # '513030': '德国etf',
    '159980': '有色金属etf',
    '513010': '恒生科技',
    # '159740': '恒生科技2',
    # '513520': '日本etf',
    # '159819':'人工智能etf',
    '159892': '恒生医药',
    '159934': '黄金etf',
    '159985': '豆粕etf',
    '510880': '红利ETF',
    # '513120': '创新药',
    '516780': '稀土',
    # '588080': '科创50',
    # '511010': '上证5年期国债ETF'
}

china_data = {}
for code, name in china_etfs.items():
    print(f"获取{name}({code})数据...")
    df = fetch_etf_data_ak(code, start_date, end_date)
    if df is not None and not df.empty:
        china_data[code] = df

# 合并所有ETF数据
all_etfs = {}
for ticker, df in intl_data.items():
    all_etfs[ticker] = df
    
for code, df in china_data.items():
    all_etfs[code] = df

# 确保所有数据都有相同的日期索引
all_dates = pd.date_range(start_date, end_date)
for ticker in all_etfs:
    all_etfs[ticker] = all_etfs[ticker].reindex(all_dates).fillna(method='ffill')

# 标准化标普500数据
sp500 = sp500.reindex(all_dates).fillna(method='ffill')


def quarterly_rotation_backtest(etf_data, start_date, end_date, initial_capital=100000):
    """
    季度再平衡策略回测 - 完全避免模糊的真值判断
    """
    # 调整开始日期，去掉第一周数据
    adjusted_start_date = start_date + timedelta(days=7)
    
    # 创建季度末日期序列
    quarters = pd.date_range(adjusted_start_date, end_date, freq='Q')
    if end_date not in quarters:
        quarters = quarters.append(pd.DatetimeIndex([end_date]))
    
    # 初始化
    portfolio_value = pd.Series(index=pd.date_range(adjusted_start_date, end_date), dtype=float)
    portfolio_value.iloc[0] = initial_capital
    current_holdings = {}  # {ticker: shares}
    current_cash = initial_capital
    
    quarterly_selections = []
    quarterly_performance = []
    
    for i in range(len(quarters)-1):
        quarter_start = quarters[i]
        quarter_end = quarters[i+1]
        
        # 计算过去一个季度的收益率
        prev_quarter_start = quarter_start - pd.offsets.QuarterEnd()
        returns = {}
        
        for ticker, df in etf_data.items():
            # 获取价格数据 - 使用显式检查
            try:
                quarter_start_data = df.loc[quarter_start]
                prev_quarter_start_data = df.loc[prev_quarter_start]
                
                # 确保我们获取的是标量值
                quarter_start_close = quarter_start_data['close']
                if isinstance(quarter_start_close, (pd.Series, pd.DataFrame)):
                    quarter_start_close = quarter_start_close.iloc[0]
                
                prev_quarter_start_close = prev_quarter_start_data['close']
                if isinstance(prev_quarter_start_close, (pd.Series, pd.DataFrame)):
                    prev_quarter_start_close = prev_quarter_start_close.iloc[0]
                
                # 显式检查NaN值
                if not np.isnan(quarter_start_close) and not np.isnan(prev_quarter_start_close):
                    returns[ticker] = (quarter_start_close - prev_quarter_start_close) / prev_quarter_start_close
            except (KeyError, AttributeError):
                continue
        
        # 选择收益率最低的4只ETF
        if len(returns) >= 4:
            selected = sorted(returns.items(), key=lambda x: x[1])[:4]
            selected_tickers = [x[0] for x in selected]
            selected_names = [intl_etfs.get(ticker, china_etfs.get(ticker, ticker)) for ticker in selected_tickers]
            quarterly_selections.append((quarter_start, selected_tickers, selected_names))
        else:
            selected_tickers = []
            quarterly_selections.append((quarter_start, [], ["现金"]))
        
        # 计算调仓日价格
        rebalance_prices = {}
        for ticker in selected_tickers:
            try:
                price_data = etf_data[ticker].loc[quarter_start]
                price = price_data['close']
                if isinstance(price, (pd.Series, pd.DataFrame)):
                    price = price.iloc[0]
                if not np.isnan(price):
                    rebalance_prices[ticker] = price
            except (KeyError, AttributeError):
                continue
        
        # 卖出不在新选择中的持仓
        to_sell = set(current_holdings.keys()) - set(selected_tickers)
        for ticker in to_sell:
            try:
                price_data = etf_data[ticker].loc[quarter_start]
                price = price_data['close']
                if isinstance(price, (pd.Series, pd.DataFrame)):
                    price = price.iloc[0]
                if not np.isnan(price):
                    current_cash += current_holdings[ticker] * price
                    del current_holdings[ticker]
            except (KeyError, AttributeError):
                continue
        
        # 计算当前总价值
        total_value = current_cash
        for ticker, shares in current_holdings.items():
            try:
                price_data = etf_data[ticker].loc[quarter_start]
                price = price_data['close']
                if isinstance(price, (pd.Series, pd.DataFrame)):
                    price = price.iloc[0]
                if not np.isnan(price):
                    total_value += shares * price
            except (KeyError, AttributeError):
                continue
        
        # 买入新增的ETF
        to_buy = [t for t in (set(selected_tickers) - set(current_holdings.keys())) 
                 if t in rebalance_prices]
        if to_buy:
            allocation = total_value / len(selected_tickers)
            
            # 调整现有持仓
            for ticker in set(current_holdings.keys()) & set(selected_tickers):
                if ticker in rebalance_prices:
                    current_value = current_holdings[ticker] * rebalance_prices[ticker]
                    delta = allocation - current_value
                    if delta > 0:
                        shares_to_buy = delta / rebalance_prices[ticker]
                        current_holdings[ticker] += shares_to_buy
                        current_cash -= shares_to_buy * rebalance_prices[ticker]
                    elif delta < 0:
                        shares_to_sell = -delta / rebalance_prices[ticker]
                        current_holdings[ticker] -= shares_to_sell
                        current_cash += shares_to_sell * rebalance_prices[ticker]
            
            # 买入新增ETF
            for ticker in to_buy:
                if ticker in rebalance_prices:
                    shares = allocation / rebalance_prices[ticker]
                    current_holdings[ticker] = shares
                    current_cash -= shares * rebalance_prices[ticker]
        
        # 如果没有新增的ETF，只需再平衡现有持仓
        elif current_holdings:
            allocation = total_value / len(current_holdings)
            
            for ticker in current_holdings:
                if ticker in rebalance_prices:
                    current_value = current_holdings[ticker] * rebalance_prices[ticker]
                    delta = allocation - current_value
                    if delta > 0:
                        shares_to_buy = delta / rebalance_prices[ticker]
                        current_holdings[ticker] += shares_to_buy
                        current_cash -= shares_to_buy * rebalance_prices[ticker]
                    elif delta < 0:
                        shares_to_sell = -delta / rebalance_prices[ticker]
                        current_holdings[ticker] -= shares_to_sell
                        current_cash += shares_to_sell * rebalance_prices[ticker]
        
        # 计算本季度每日组合价值
        quarter_values = []
        for date in pd.date_range(quarter_start, quarter_end):
            if date in portfolio_value.index:
                daily_value = current_cash
                for ticker, shares in current_holdings.items():
                    try:
                        price_data = etf_data[ticker].loc[date]
                        price = price_data['close']
                        if isinstance(price, (pd.Series, pd.DataFrame)):
                            price = price.iloc[0]
                        if not np.isnan(price):
                            daily_value += shares * price
                    except (KeyError, AttributeError):
                        continue
                portfolio_value.loc[date] = daily_value
                quarter_values.append(daily_value)
        
        # 计算本季度收益率
        if len(quarter_values) > 1 and quarter_values[0] != 0:
            quarter_return = (quarter_values[-1] - quarter_values[0]) / quarter_values[0]
        else:
            quarter_return = 0
        quarterly_performance.append((quarter_start, quarter_return))
    
    portfolio_value = portfolio_value.ffill()
    return portfolio_value, quarterly_selections, quarterly_performance
    

# 执行回测
portfolio_value, quarterly_selections, quarterly_performance = quarterly_rotation_backtest(all_etfs, start_date, end_date)

# 计算每日收益率
daily_returns = portfolio_value.pct_change().dropna()

# ================== 绩效计算部分 ==================

def calculate_performance_metrics(portfolio_value, daily_returns, benchmark_returns):
    """
    计算绩效指标
    :param portfolio_value: 组合价值序列
    :param daily_returns: 每日收益率序列
    :param benchmark_returns: 基准收益率序列
    :return: 绩效指标字典
    """
    # 总收益率
    total_return = (portfolio_value.iloc[-1] / portfolio_value.iloc[0]) - 1
    
    # 年化收益率
    years = len(portfolio_value) / 252
    annualized_return = (1 + total_return) ** (1/years) - 1
    
    # 年化波动率
    annualized_volatility = daily_returns.std() * np.sqrt(252)
    
    # 夏普比率(假设无风险利率3%)
    risk_free_rate = 0.03
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    
    # 最大回撤
    cumulative_returns = (1 + daily_returns).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()
    
    # 确保日期对齐
    aligned_returns, aligned_benchmark = daily_returns.align(benchmark_returns, join='inner')
    aligned_returns_array = aligned_returns.values
    aligned_benchmark_array = aligned_benchmark.values
    
    # 检查并调整数组形状
    if aligned_returns_array.ndim > 1:
        aligned_returns_array = aligned_returns_array.flatten()
    if aligned_benchmark_array.ndim > 1:
        aligned_benchmark_array = aligned_benchmark_array.flatten()
    
    # 计算Beta
    covariance_matrix = np.cov(aligned_returns_array, aligned_benchmark_array)
    covariance = covariance_matrix[0, 1]
    benchmark_variance = np.var(aligned_benchmark_array)
    beta = covariance / benchmark_variance
    
    # 计算Alpha - 确保结果是标量值
    benchmark_annual_return = (1 + aligned_benchmark.mean()) ** 252 - 1
    if isinstance(benchmark_annual_return, pd.Series):
        benchmark_annual_return = benchmark_annual_return.iloc[0]
    alpha = float(annualized_return - (risk_free_rate + beta * (benchmark_annual_return - risk_free_rate)))
    
    # 分年度收益率
    yearly_returns = portfolio_value.resample('Y').last().pct_change()
    yearly_returns.index = yearly_returns.index.year
    yearly_returns = yearly_returns.dropna()
    
    return {
        '总收益率': total_return,
        '年化收益率': annualized_return,
        '年化波动率': annualized_volatility,
        '夏普比率': sharpe_ratio,
        '最大回撤': max_drawdown,
        'Alpha': alpha,
        'Beta': beta,
        '分年度收益率': yearly_returns
    }

# 计算标普500收益率
sp500_returns = sp500['close'].pct_change().dropna()

# 计算绩效指标
metrics = calculate_performance_metrics(portfolio_value, daily_returns, sp500_returns)

# ================== 结果展示部分 ==================

# print("\n季度选择记录:")
# for quarter in quarterly_selections:
#     date = quarter[0].strftime('%Y-%m-%d')
#     tickers = ", ".join(quarter[1])
#     names = ", ".join(quarter[2])
#     print(f"{date}: 选择基金 {tickers} ({names})")

print("\n季度选择记录及收益率:")
for i, quarter in enumerate(quarterly_selections):
    date = quarter[0].strftime('%Y-%m-%d')
    tickers = ", ".join(quarter[1]) if quarter[1] else "现金"
    names = ", ".join(quarter[2]) if quarter[2] else "现金"
    # 修改这里，因为quarter只有3个元素
    q_return = quarterly_performance[i][1] * 100
    print(f"{date}: 选择基金 {tickers} ({names}) | 季度收益率: {q_return:.2f}%")

print("\n策略绩效指标(5年回测):")
print(f"初始资金: 100,000元")
print(f"最终资金: {portfolio_value.iloc[-1]:,.2f}元")
print(f"总收益率: {metrics['总收益率']*100:.2f}%")
print(f"年化收益率: {metrics['年化收益率']*100:.2f}%")
print(f"年化波动率: {metrics['年化波动率']*100:.2f}%")
print(f"夏普比率: {metrics['夏普比率']:.2f}")
print(f"最大回撤: {metrics['最大回撤']*100:.2f}%")
print(f"Alpha(相对于标普500): {metrics['Alpha']*100:.2f}%")
print(f"Beta(相对于标普500): {metrics['Beta']:.2f}")

print("\n分年度收益率:")
for year, ret in metrics['分年度收益率'].items():
    print(f"{year}: {ret*100:.2f}%")

# 绘制组合价值曲线
plt.figure(figsize=(14, 7))
(portfolio_value / 100000).plot(label='策略净值(10万初始)', linewidth=2)

# 绘制标普500作为比较
(sp500['close'] / sp500['close'].iloc[0]).plot(label='标普500', linestyle='--')

plt.title('ETF季度反转策略(4只基金) vs 标普500 (标准化后)', fontsize=15)
plt.xlabel('日期', fontsize=12)
plt.ylabel('净值(标准化)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

获取QQQ数据...


[*********************100%***********************]  1 of 1 completed


获取SPY数据...


[*********************100%***********************]  1 of 1 completed


获取标普500指数数据...


[*********************100%***********************]  1 of 1 completed


获取有色金属etf(159980)数据...
获取恒生科技(513010)数据...
获取恒生医药(159892)数据...
获取黄金etf(159934)数据...
获取豆粕etf(159985)数据...
获取红利ETF(510880)数据...
获取稀土(516780)数据...

季度选择记录及收益率:
2021-09-30: 选择基金 现金 (现金) | 季度收益率: 0.00%
2021-12-31: 选择基金 513010, 510880, 159985, 159934 (恒生科技, 红利ETF, 豆粕etf, 黄金etf) | 季度收益率: 5.08%
2022-03-31: 选择基金 159892, 513010, 516780, QQQ (恒生医药, 恒生科技, 稀土, 纳斯达克ETF) | 季度收益率: 3.17%
2022-06-30: 选择基金 QQQ, 159980, SPY, 159985 (纳斯达克ETF, 有色金属etf, 标普500ETF, 豆粕etf) | 季度收益率: 1.06%
2022-09-30: 选择基金 513010, 159892, 516780, SPY (恒生科技, 恒生医药, 稀土, 标普500ETF) | 季度收益率: 14.20%
2022-12-31: 选择基金 510880, 516780, QQQ, 159934 (红利ETF, 稀土, 纳斯达克ETF, 黄金etf) | 季度收益率: 9.17%
2023-03-31: 选择基金 159985, 159892, 159980, 513010 (豆粕etf, 恒生医药, 有色金属etf, 恒生科技) | 季度收益率: -2.90%
2023-06-30: 选择基金 159892, 513010, 516780, 159980 (恒生医药, 恒生科技, 稀土, 有色金属etf) | 季度收益率: -2.49%
2023-09-30: 选择基金 516780, SPY, QQQ, 513010 (稀土, 标普500ETF, 纳斯达克ETF, 恒生科技) | 季度收益率: 4.55%
2023-12-31: 选择基金 516780, 159985, 513010, 510880 (稀土, 豆粕etf, 恒生科技, 红利ETF) | 季度收益率: -0.37

ModuleNotFoundError: No module named 'tkinter'